In [1]:
# !conda config --prepend channels conda-forge
# !conda create -n ox --strict-channel-priority osmnx # run this if you haven't installed osmnx
!pip install xmltodict
!pip install geopandas
!pip install imagehash
!pip install osmnx
!pip install gluoncv
!pip install rasterio
!pip install lightgbm
!pip install joblib
!pip install sklearn
# !pip install selenium
# !pip install webdriver_manager
# !pip install inplace-abn
# local python script
from Perception_Prediction import pipeline_utils
from get_gsv import get_gsv
from get_aqi import get_aqi
from stitch_image import stitch_images
from Classification import classification
from Segmentation import segmentation
from Detection import detection
from Edge_Detection import edge_detection
from Blob_Detection import blob_detection
from HLS_Statistics import hls_statistics
from image_hash import image_hash

# non-local libraries
import glob
import pandas as pd
import geopandas as gpd
import shutil, os
import random
import sagemaker
from sagemaker import get_execution_role
import boto3
import tqdm
from PIL import Image
from IPython.display import display
import imagehash
import time
import sys
import numpy as np
from shapely.geometry import Point
from geopandas.tools import sjoin
import osmnx as ox
import json
from pandas.io.json import json_normalize #package for flattening json in pandas df
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import fiona
import math
import rasterio
# from rasterio.plot import show
from sklearn.model_selection import train_test_split
import sklearn
import joblib
import seaborn as sns
import copy 
import scipy as sp
from scipy.stats import chi2
from sklearn.covariance import MinCovDet
import matplotlib.pyplot as plt
from scipy import stats
from collections import Counter
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score
import lightgbm as lgb
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

     |████████████████████████████████| 804.1 MB 854 bytes/s a 0:00:01     |█████████                       | 226.8 MB 55.7 MB/s eta 0:00:11
     |████████████████████████████████| 17.4 MB 15.3 MB/s eta 0:00:01


# Table of content

1. Get Street Network from OSM
2. Create Sample Points from Street Network
3. Sample 8,000 points (To reduce the computation when collecting meta data)
4. Collect GSV meta data
5. Get GSV images
6. Stitch images
7. Remove grey images
8. Sample 1,500 images for survey, save locally, and upload them to S3 bucket
9. Get URLs of survey images
10. Feature extraction

# Get Street Network from OSM

In [ ]:
# city_list=['Singapore','Tokyo']
city_list=[
    'Chiyoda, Tokyo',
    'Chuo, Tokyo',
    'Minato, Tokyo',
    'Shinjuku, Tokyo',
    'Bunkyo, Tokyo',
    'Taito, Tokyo',
    'Sumida, Tokyo',
    'Koto, Tokyo',
    'Shinagawa, Tokyo',
    'Meguro, Tokyo',
    'Ota, Tokyo',
    'Setagaya, Tokyo',
    'Shibuya, Tokyo',
    'Nakano, Tokyo',
    'Suginami, Tokyo',
    'Toshima, Tokyo',
    'Kita, Tokyo',
    'Arakawa, Tokyo',
    'Itabashi, Tokyo',
    'Nerima, Tokyo',
    'Adachi,, Tokyo',
    'Katsushika, Tokyo',
    'Edogawa, Tokyo'
]
root = "/home/ec2-user/SageMaker/"
out_folder=os.path.join(root,'data/network/')
get_gsv.getStreetNetwork(city_list,out_folder)

# Create Sample Points from Street Network

In [ ]:
# Combine all the Tokyo shp (edges)
root = "/home/ec2-user/SageMaker/"
tokyo_shp_list = glob.glob(os.path.join(root,'data/network/*, Tokyo_street_network/edges.shp'))
gdf = gpd.GeoDataFrame(pd.concat([gpd.read_file(i) for i in tokyo_shp_list],
                                 ignore_index=True), crs=gpd.read_file(tokyo_shp_list[0]).crs)
if not os.path.exists(os.path.join(root,'data/network/Tokyo_street_network')):
    os.makedirs(os.path.join(root,'data/network/Tokyo_street_network'))
gdf.to_file(os.path.join(root,'data/network/Tokyo_street_network/edges.shp'))

In [ ]:
# Combine all the Tokyo shp (nodes)
root = "/home/ec2-user/SageMaker/"
tokyo_shp_list = glob.glob(os.path.join(root,'data/network/*, Tokyo_street_network/nodes.shp'))
gdf = gpd.GeoDataFrame(pd.concat([gpd.read_file(i) for i in tokyo_shp_list],
                                 ignore_index=True), crs=gpd.read_file(tokyo_shp_list[0]).crs)
if not os.path.exists(os.path.join(root,'data/network/Tokyo_street_network')):
    os.makedirs(os.path.join(root,'data/network/Tokyo_street_network'))
gdf.to_file(os.path.join(root,'data/network/Tokyo_street_network/nodes.shp'))

In [ ]:
# create sample points on the street network
city_list=[
#     'Singapore',
    'Tokyo']
root = "/home/ec2-user/SageMaker/"
outfolder=os.path.join(root,'data/point_data/')
if not os.path.exists(outfolder):
    os.makedirs(outfolder)
for city in city_list:
    mini_dist = 1000 #the minimum distance between two generated points in meters
    inshp = os.path.join(root,'data/network/{}_street_network/edges.shp'.format(city)) #the input shapefile of road network
    outshp = os.path.join(outfolder,'{}_{}m_point.shp'.format(city,mini_dist)) #the output shapefile of the points
    get_gsv.createPoints(inshp, outshp, mini_dist)

In [ ]:
# check the points created
city_list=['Singapore','Tokyo']
root = "/home/ec2-user/SageMaker/"
outfolder=os.path.join(root,'data/point_data/')
mini_dist=1000
for city in city_list:
    points=gpd.read_file(os.path.join(root,outfolder,'{}_{}m_point.shp'.format(city,mini_dist)))
    print(city, points.info())

# Sample 8,000 points (To reduce the computation when collecting meta data)

In [ ]:
city_list=[
#     'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"
sample_num=8000
for city in city_list:
    inshp=os.path.join(root,'data/point_data/{}_1000m_point.shp'.format(city))
    outshp=os.path.join(root,'data/point_data/{}_{}_sample_point.shp'.format(city,str(sample_num)))
    get_gsv.getSamplePoints(inshp,outshp,sample_num)

# Collect GSV meta data

In [ ]:
city_list=[
#     'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"
sample_num=8000
for city in city_list:
    inputShp =os.path.join(root,'data/point_data/{}_{}_sample_point.shp'.format(city,str(sample_num)))
    outputTxt = os.path.join(root,'data/meta_data_{}/'.format(city))
    batch_num=1000
    get_gsv.GSVpanoMetadataCollector(inputShp,batch_num,outputTxt)

In [ ]:
# put all the meta data files together
city_list=[
#     'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"
for city in city_list:
    file_list = glob.glob(os.path.join(root,'data/meta_data_{}/Pnt*.csv'.format(city)))
    df = pd.DataFrame(pd.concat([pd.read_csv(i) for i in file_list],
                                 ignore_index=True))
    df.to_csv(os.path.join(root,'data/meta_data_{}/meta_data_{}.csv'.format(city,city)))

# Sample 7,142 Points (Because this is the max number with free tier)

In [ ]:
city_list=[
#     'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"
sample_num=7142

# create an output folder if it doesnt exist
outfolder=os.path.join(root,'data/pano_id_list/')
if not os.path.exists(outfolder):
    os.makedirs(outfolder)
    
for city in city_list:
    intxt=os.path.join(root,'data/meta_data_{}/meta_data_{}.csv'.format(city,city))
    outtxt=os.path.join(outfolder,'{}_{}_sample_point.csv'.format(city,str(sample_num)))
    get_gsv.getSamplePoints(intxt,outtxt,sample_num)

# Get GSV images (!Don't run this twice!)
## Also, dont forget to exclude indoor images

In [ ]:
city_list=[
#     'Singapore',
#     'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"
sample_num=7142
    
for city in city_list:
    # create outfolder
    outputImgFolder=os.path.join(root,'data/img_seperate_{}/'.format(city))
    # prevent the second run
    if os.path.exists(outputImgFolder):
        continue
    inputCSV = os.path.join(root,'data/pano_id_list/{}_{}_sample_point.csv'.format(city,str(sample_num)))
    get_gsv.getGSV(inputCSV,outputImgFolder)

# Stitch the images

In [ ]:
city_list=[
#     'Singapore'
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"
sample_num=7142

for city in city_list:
    inputCSV = os.path.join(root,'data/pano_id_list/{}_{}_sample_point.csv'.format(city,str(sample_num)))
    inputImgFolder = os.path.join(root,'data/img_seperate_{}/'.format(city))
    outputImgFolder = os.path.join(root,'data/img_stitched_{}/'.format(city))
    stitch_images.stitchImg(inputCSV,inputImgFolder,outputImgFolder)

# remove gray images

In [ ]:
# find gray images by using imagehash and move them to a seperate folder
city_list=[
#     'Singapore'
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"
gray_img=os.path.join(root,'data/gray_img/pano=CAoSLEFGMVFpcE0tTkFGRmEtOXdibWE4N2FBY1dxLXV0RURxVzRiXzhOOFlPSEdp.jpg')
hash = imagehash.average_hash(Image.open(gray_img))
for city in city_list:
    outfolder=os.path.join(root,'data/gray_img_{}/'.format(city))
    if not os.path.exists(outfolder):
        os.makedirs(outfolder)
    img_list=glob.glob(os.path.join(root,'data/img_stitched_{}/*.jpg'.format(city)))
    for img in tqdm.tqdm(img_list):
        otherhash = imagehash.average_hash(Image.open(img))
        if hash == otherhash:
            shutil.move(img,os.path.join(outfolder,img.split('/')[-1]))

In [ ]:
# check images in "gray_img_..." folder
city_list=[
#     'Singapore'
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

for city in city_list:
    gray_img_list=glob.glob(os.path.join(root,'data/gray_img_{}/*.jpg'.format(city)))
    for gray_img in tqdm.tqdm(gray_img_list):
        pil_im = Image.open(gray_img)
        display(pil_im)

# sample 1,500 images for survey, save locally, and upload them to S3 bucket

In [ ]:
city_list=[
    'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"
n=1500 # sample size

# set up sagemaker session
role = get_execution_role()
print(role)
sess = sagemaker.Session()
print(sess)

# create a s3 bucket
bucket_name='gsv-perception-survey-img'
key_prefix='data'
s3 = boto3.resource('s3')
my_region = boto3.session.Session().region_name
try:
    if  my_region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

# go through city_list
for city in city_list:
    # sample 1,500 images
    img_list=glob.glob(os.path.join(root,'data/img_stitched_{}/*.jpg'.format(city)))
    sampled_img=random.sample(img_list, n)
    # create a local output folder
    out_folder=os.path.join(root,'data/img_survey/{}/'.format(city))
    if not os.path.exists(out_folder):
        os.makedirs(out_folder)
    
    # save in img_survey folder and upload the data to bucket
    for img in tqdm.tqdm(sampled_img):
        # save locally
        shutil.copy2(img, out_folder)
        
        # upload to s3
        sess.upload_data(path=img, bucket=bucket_name,key_prefix=key_prefix)

# Get URL of the Survey Images in S3

In [ ]:
# set up necessary variables
root = "/home/ec2-user/SageMaker/"
s3 = boto3.resource('s3')
bukect_str='gsv-perception-survey-img'
my_bucket = s3.Bucket(bukect_str)
url_list=[]
outputFolder=os.path.join(root,'data/tabular_data/')

# go through all the objects in the bucket
for file in my_bucket.objects.all():
    # modify file name for URL encoding
    file_modified=file.key.replace('=','%3D')
    # create url
    url='https://{BUCKET_NAME}.s3-ap-southeast-1.amazonaws.com/{FILE_NAME}'.\
    format(BUCKET_NAME=bukect_str,FILE_NAME=file_modified)
    url_list.append(url)
url_df=pd.DataFrame(url_list,columns=['url'])
url_df.to_csv(os.path.join(outputFolder,'aws_url.csv'), index=False)

In [ ]:
# # don't run this unless you want to delete all the sample images in a s3 bukect for surveys
# import boto3
# s3 = boto3.resource('s3')
# bucket = s3.Bucket('gsv-perception-survey-img')
# for obj in tqdm.tqdm(bucket.objects.filter(Prefix='data/')):
#     s3.Object(bucket.name,obj.key).delete()

# Upload the remaining stitched images to S3

In [ ]:
city_list=[
#     'Singapore'
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# set up sagemaker session
role = get_execution_role()
print(role)
sess = sagemaker.Session()
print(sess)

# create a s3 bucket
bucket_name='gsv-img'
s3 = boto3.resource('s3')
my_region = boto3.session.Session().region_name
try:
    if  my_region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
    time.sleep(5)
except Exception as e:
    print('S3 error: ',e)
#     sys.exit()  
    

# go through city_list
for city in city_list:
    img_list=glob.glob(os.path.join(root,'data/img_stitched_{}/*.jpg'.format(city)))
    # upload the data to bucket
    for img in tqdm.tqdm(img_list,position=0):
        sess.upload_data(path=img, bucket=bucket_name,key_prefix=city)

# Get locations of Remaining Stitched Images

In [ ]:
city_list=[
    'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# go through city_list
for city in city_list:
    # get meta data with pano IDs and locations
    meta_data=os.path.join(root,'data/meta_data_{}/meta_data_{}.csv'.format(city,city))
    meta_data_df=pd.read_csv(meta_data)
    
    # get all the remaining images' pano IDs and convert to dataframe
    img_list=glob.glob(os.path.join(root,'data/img_stitched_{}/*.jpg'.format(city)))
    img_list_len=len(img_list)
    pano_id_list=[os.path.split(img)[1].replace('.jpg','').replace('pano=','') for img in img_list]
    pano_id_df = pd.DataFrame(pano_id_list,columns=['panoId'])
    img_location_df=pd.merge(pano_id_df,
                             meta_data_df[['panoId','panoLon','panoLat','distDiff']],
                             on=['panoId'],
                             how='left')
    
    # since there are some duplicates of panoIds in the meta data, drop them
    img_location_df=img_location_df.drop_duplicates(subset=['panoId'])
    
    # save to meta data folder
    img_location_df.to_csv(os.path.join(root,'data/meta_data_{}/{}_remaining_points.csv'.format(city,city)))        

# Extract non-perception features 

## Segmentation
- quality of pavement (pothole)
- quality of street lighting
- presence of bike lanes
- Street amenities (Garbage cans, Benches, phone booth, )
- Utility pole
- Bike parking
- Directional signs
- presence of sidewalk
- Presense and quality of crosswalk (with/without traffic lights, traffic signs)
- street accesibility (curb cuts)
- stop sign
- scenery along the bike lanes (built up area)
- scenery along the bike lanes (green area)
- scenery along the bike lanes (SVF)
- scenery along the bike lanes (water)

#### option1: deploy model here

In [ ]:
city_list=[
#     'Singapore'
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# go through city_list
for city in city_list:
    # define input variables
    model_data='s3://sagemaker-ap-southeast-1-428024436188/semantic-segmentation-demo/output/mapillary-segmentation-3/output/model.tar.gz'
    input_img_folder=os.path.join(root,'data/img_stitched_{}/'.format(city))
    resized_img_folder=os.path.join(root,'data/img_data_resized_{}/'.format(city))
    output_txt_folder=os.path.join(root,'data/tabular_data/{}/'.format(city))
    output_img_folder=os.path.join(root,'output/segmentation/{}/').format(city)
    class_json_file=os.path.join(root,'models/config_v2.0.json')
    for variable in [resized_img_folder,output_txt_folder]:
        if not os.path.exists(variable):
            os.makedirs(variable)
    segmentation.segment_images(model_data,input_img_folder,resized_img_folder,output_txt_folder,output_img_folder,class_json_file,s3_upload_bucket=False)

#### option2: run a pretrained model

##### singapore

In [ ]:
%cd /home/ec2-user/SageMaker/inplace_abn/scripts/
!python test_vistas.py /home/ec2-user/SageMaker/models/wide_resnet38_deeplab_vistas.pth.tar /home/ec2-user/SageMaker/data/img_stitched_Singapore /home/ec2-user/SageMaker/data/segmentation_test

In [ ]:
# copy the segmented images from segmentation_test to output/segmentation/Singapore
root = "/home/ec2-user/SageMaker/"
segmented_img_list=glob.glob(os.path.join(root,'data/segmentation_test/*.png'))
for segmented_img in segmented_img_list:
    shutil.copy2(segmented_img,os.path.join(root,'output/segmentation/Singapore'))

##### Tokyo

In [ ]:
%cd /home/ec2-user/SageMaker/inplace_abn/scripts/
!python test_vistas.py /home/ec2-user/SageMaker/models/wide_resnet38_deeplab_vistas.pth.tar /home/ec2-user/SageMaker/data/img_stitched_Tokyo /home/ec2-user/SageMaker/output/segmentation/Tokyo

#### Extract pixel proportions

In [ ]:
city_list=[
    'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# import classes as a dataframe
root = "/home/ec2-user/SageMaker/"
config=os.path.join(root,'models/config_v1.2.json')
with open(config) as f:
    d = json.load(f)
classes_df = json_normalize(d['labels'],max_level=3)

# go through city_list
for city in city_list:
    # create output_folder
    output_folder=os.path.join(root,'data/tabular_data/{}/'.format(city))
    if not os.path.exists(output_folder):
            os.makedirs(output_folder)
            
    # creating variables to be used for extracting the pixl values
    result_dict={}
    
    # segmented image list
    segmented_img_list=glob.glob(os.path.join(root,'output/segmentation/{}/*.png'.format(city)))
    for segmented_img in tqdm.tqdm(segmented_img_list):
        temp_dict={}
        im = Image.open(segmented_img)
        array=np.array(im)
        # calculate each category's pixel proportion
        for i in range(len(classes_df)):
            total_num=array.shape[0]*array.shape[1]
            x_num=np.count_nonzero(array==i)
            ratio=x_num/total_num
            temp_dict[i]=ratio
        result_dict[os.path.split(segmented_img)[1].replace('.png','')]= temp_dict
    result_df=pd.DataFrame(result_dict)
    result_df=result_df.transpose()

    # change the column names
    result_df.columns=classes_df['name'].tolist()
    result_df.to_csv(os.path.join(output_folder,'segmentation.csv'))

#### Extract indicators
- quality of pavement (pothole)
- quality of street lighting
- presence of bike lanes
- Street amenities (Garbage cans, Benches, phone booth, )
- Utility pole
- Bike parking
- Directional signs
- presence of sidewalk
- Presense and quality of crosswalk (with/without traffic lights, traffic signs)
- street accesibility (curb cuts)
- stop sign
- scenery along the bike lanes (built up area)
- scenery along the bike lanes (green area)
- scenery along the bike lanes (SVF)
- scenery along the bike lanes (water)

In [ ]:
# a list of indicators
indicator_dict={'pavement':['object--pothole'],
                'street_light':['object--street-light'],
                'bike_lanes':['construction--flat--bike-lane'],
                'street_amenity':['object--trash-can',
                                  'object--bench',
                                  'object--phone-booth'],
                'utility_pole':['object--support--utility-pole'],
                'bike_parking':['object--bike-rack'],
                'side_walk':['construction--flat--sidewalk'],
                'cross_walk':['construction--flat--crosswalk-plain',
                             'marking--crosswalk-zebra'],
                'accessibility':['construction--flat--curb-cut'],
                'scenery_building':['construction--structure--building'],
                'scenery_greenery':['nature--vegetation'],
                'scenery_sky':['nature--sky'],
                'scenery_water':['nature--water']
               }
city_list=[
    'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# go through city_list
for city in city_list:
    segmentation_result=os.path.join(root,'data/tabular_data/{}/segmentation.csv'.format(city))
    segmentation_result_df=pd.read_csv(segmentation_result)
    # rename the first col to pano_id
    segmentation_result_df=segmentation_result_df.rename(columns={'Unnamed: 0':'pano_id'})
    for key in indicator_dict.keys():
        temp_df=segmentation_result_df[['pano_id']]
        for indicator in indicator_dict[key]:
            indicator_df=segmentation_result_df[['pano_id',indicator]]
            # append indicator_df to temp_df
            temp_df=pd.merge(temp_df,
                             indicator_df,
                             on='pano_id',
                             how='left'
                            )
        temp_df['total']=temp_df.iloc[:,1:].sum(axis=1)
        
        temp_df.to_csv(os.path.join(root,'data/tabular_data/{}/{}.csv'.format(city,key)))

## Detection
### cycling mode share
    0. Filter detection data to only have transport related columns
    1. Merge locational data with the detection data
    2. Copy the dataframe
    3. Conduct spatial aggregation with 500m buffer between the original and the copy
    4. Group by pano_id
    5. Calculate the sum of all the columns
    6. Calculate the cycling mode share

In [ ]:
city_list=[
    'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# go through city_list
for city in city_list:
    # detection result
    detection_result=os.path.join(root,'data/tabular_data/{}/detection.csv'.format(city))
    detection_result_df=pd.read_csv(detection_result)
    detection_result_df=detection_result_df.rename(columns={'Unnamed: 0':'pano_id'})
    detection_result_df_transport=detection_result_df[['pano_id','person','bicycle','car','motorcycle','bus','train','truck']]
    
    # location data
    location=os.path.join(root,'data/meta_data_{}/{}_remaining_points.csv'.format(city,city))
    location_df=pd.read_csv(location)
    location_df['geometry']=location_df.apply(lambda x: Point((float(x.panoLon), float(x.panoLat))), axis=1)
    location_gdf=gpd.GeoDataFrame(location_df, geometry='geometry')
    location_gdf=location_gdf.set_crs(epsg=4326)
    #convert to Pseudo-Mercator
    location_gdf=location_gdf.to_crs("EPSG:3857")
    
    # merge location to detection result to convert it to gdf
    location_gdf['panoId']='pano='+location_gdf['panoId']
    detection_result_df_transport=pd.merge(detection_result_df_transport,
                                           location_gdf[['panoId','geometry']],
                                           left_on='pano_id',
                                           right_on='panoId',
                                           how='left')
    detection_gdf=gpd.GeoDataFrame(detection_result_df_transport, geometry='geometry')
    
    # create 500m buffer around img_location_gdf
    buffer_500m = location_gdf.copy()
    buffer_500m["geometry"]=location_gdf.geometry.buffer(500)
    # coduct spatial join to extract points that intersect with the 500m buffer
    detection_in_500m = gpd.sjoin(buffer_500m, detection_gdf, how="left", op='intersects')
    
    # group by panoId to get sum
    detection_in_500m_grouped=detection_in_500m.groupby(['pano_id']).\
                              agg({'person':'sum',
                                   'bicycle':'sum',
                                   'car':'sum',
                                   'motorcycle':'sum',
                                   'bus':'sum',
                                   'train':'sum',
                                   'truck':'sum'}).\
                              reset_index()
    detection_in_500m_grouped['total']=detection_in_500m_grouped.iloc[:,1:].sum(axis=1)
    detection_in_500m_grouped['bicycle_share']=detection_in_500m_grouped['bicycle']/detection_in_500m_grouped['total']
    detection_in_500m_grouped['bicycle_share']=detection_in_500m_grouped['bicycle_share'].fillna(0)
    detection_in_500m_grouped=detection_in_500m_grouped[['pano_id','bicycle_share']]
    print(detection_in_500m_grouped)
    detection_in_500m_grouped.to_csv(os.path.join(root,'data/tabular_data/{}/bicycle_share.csv'.format(city)))

### No. of Vehicles
    0. Filter detection data to only have transport related columns
    1. Merge locational data with the detection data
    2. Copy the dataframe
    3. Conduct spatial aggregation with 500m buffer between the original and the copy
    4. Group by pano_id
    5. Calculate the sum of all the columns
    6. Calculate the cycling mode share

In [ ]:
city_list=[
    'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# go through city_list
for city in city_list:
    # detection result
    detection_result=os.path.join(root,'data/tabular_data/{}/detection.csv'.format(city))
    detection_result_df=pd.read_csv(detection_result)
    detection_result_df=detection_result_df.rename(columns={'Unnamed: 0':'pano_id'})
    detection_result_df_transport=detection_result_df[['pano_id','car','motorcycle','bus','truck']]
    
    # location data
    location=os.path.join(root,'data/meta_data_{}/{}_remaining_points.csv'.format(city,city))
    location_df=pd.read_csv(location)
    location_df['geometry']=location_df.apply(lambda x: Point((float(x.panoLon), float(x.panoLat))), axis=1)
    location_gdf=gpd.GeoDataFrame(location_df, geometry='geometry')
    location_gdf=location_gdf.set_crs(epsg=4326)
    #convert to Pseudo-Mercator
    location_gdf=location_gdf.to_crs("EPSG:3857")
    
    # merge location to detection result to convert it to gdf
    location_gdf['panoId']='pano='+location_gdf['panoId']
    detection_result_df_transport=pd.merge(detection_result_df_transport,
                                           location_gdf[['panoId','geometry']],
                                           left_on='pano_id',
                                           right_on='panoId',
                                           how='left')
    detection_gdf=gpd.GeoDataFrame(detection_result_df_transport, geometry='geometry')
    
    # create 500m buffer around img_location_gdf
    buffer_500m = location_gdf.copy()
    buffer_500m["geometry"]=location_gdf.geometry.buffer(500)
    # coduct spatial join to extract points that intersect with the 500m buffer
    detection_in_500m = gpd.sjoin(buffer_500m, detection_gdf, how="left", op='intersects')
    
    # group by panoId to get sum
    detection_in_500m_grouped=detection_in_500m.groupby(['pano_id']).\
                              agg({'car':'sum',
                                   'motorcycle':'sum',
                                   'bus':'sum',
                                   'truck':'sum'}).\
                              reset_index()
    detection_in_500m_grouped['no_of_vehicles']=detection_in_500m_grouped.iloc[:,1:].sum(axis=1)
    detection_in_500m_grouped=detection_in_500m_grouped[['pano_id','no_of_vehicles']]
    detection_in_500m_grouped.to_csv(os.path.join(root,'data/tabular_data/{}/no_of_vehicles.csv'.format(city)))

### stop signs/traffic lights

In [12]:
city_list=[
    'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# go through city_list
for city in city_list:
    # detection result
    detection_result=os.path.join(root,'data/tabular_data/{}/detection.csv'.format(city))
    detection_result_df=pd.read_csv(detection_result)
    detection_result_df=detection_result_df.rename(columns={'Unnamed: 0':'pano_id'})
    detection_result_df_trffic_light_stop_sign=detection_result_df[['pano_id','traffic light','stop sign']]
    
    # location data
    location=os.path.join(root,'data/meta_data_{}/{}_remaining_points.csv'.format(city,city))
    location_df=pd.read_csv(location)
    location_df['geometry']=location_df.apply(lambda x: Point((float(x.panoLon), float(x.panoLat))), axis=1)
    location_gdf=gpd.GeoDataFrame(location_df, geometry='geometry')
    location_gdf=location_gdf.set_crs(epsg=4326)
    #convert to Pseudo-Mercator
    location_gdf=location_gdf.to_crs("EPSG:3857")
    
    # merge location to detection result to convert it to gdf
    location_gdf['panoId']='pano='+location_gdf['panoId']
    detection_result_df_trffic_light_stop_sign=pd.merge(detection_result_df_trffic_light_stop_sign,
                                           location_gdf[['panoId','geometry']],
                                           left_on='pano_id',
                                           right_on='panoId',
                                           how='left')
    detection_gdf=gpd.GeoDataFrame(detection_result_df_trffic_light_stop_sign, geometry='geometry')
    
    # create 100m buffer around img_location_gdf
    buffer_100m = location_gdf.copy()
    buffer_100m["geometry"]=location_gdf.geometry.buffer(100)
    # coduct spatial join to extract points that intersect with the 100m buffer
    detection_in_100m = gpd.sjoin(buffer_100m, detection_gdf, how="left", op='intersects')
    
    # group by panoId to get sum
    detection_in_100m_grouped=detection_in_100m.groupby(['pano_id']).\
                              agg({'traffic light':'sum',
                                   'stop sign':'sum'
                                   }).\
                              reset_index()
    
    # create a column for traffic light stop sign indicator
    detection_in_100m_grouped['traffic_light_stop_sign']=0
    cond_list=[(detection_in_100m_grouped['stop sign']>0)|(detection_in_100m_grouped['traffic light']>0)]
    choice_list=[1]
    detection_in_100m_grouped['traffic_light_stop_sign']=np.select(cond_list,choice_list)
    detection_in_100m_grouped.to_csv(os.path.join(root,'data/tabular_data/{}/traffic_light_stop_sign.csv'.format(city)))

## OSM
### 500m buffer
- Number of intersection with lights
- Number of intersection without lights
- Number of cul-de-sac / dead-end
- Number of points of interest
- Number of transit facilities

#### Number of intersection with lights & Number of intersection without lights 

##### get crossing with and without traffic signals from highway nodes in OSM

In [ ]:
city_list=[
    'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# parameter for osmnx
tags={'highway':['crossing','traffic_signal']}
# go through cities in the list
for city in city_list:
    crossing=ox.geometries_from_place(city, tags)
    crossing=crossing.loc[crossing['element_type']=='node']
    crossing=crossing.drop(['nodes'],axis=1)
    crossing.to_file(os.path.join(root,'data/network/{}_street_network/{}_crossing.shp').format(city,city))

In [ ]:


city_list=[
    'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

for city in tqdm.tqdm(city_list):
    # get the remaining images' locations
    img_location=os.path.join(root,'data/meta_data_{}/{}_remaining_points.csv'.format(city,city))
    #load the csv file and convert it to gdf
    img_location_df=pd.read_csv(img_location)
    img_location_df['geometry']=img_location_df.apply(lambda x: Point((float(x.panoLon), float(x.panoLat))), axis=1)
    img_location_gdf=gpd.GeoDataFrame(img_location_df, geometry='geometry')
    img_location_gdf=img_location_gdf.set_crs(epsg=4326)
    #convert to Pseudo-Mercator
    img_location_gdf=img_location_gdf.to_crs("EPSG:3857")
    
    # get crossing shp
    crossing=os.path.join(root,'data/network/{}_street_network/{}_crossing.shp'.format(city,city))
    crossing_gdf=gpd.read_file(crossing)
    #convert to Pseudo-Mercator
    crossing_gdf=crossing_gdf.to_crs("EPSG:3857")

    # create 500m buffer around img_location_gdf
    buffer_500m = img_location_gdf.copy()
    buffer_500m["geometry"]=img_location_gdf.geometry.buffer(500)
    # coduct spatial join to extract points that intersect with the 500m buffer
    nodes_in_500m = gpd.sjoin(crossing_gdf, buffer_500m, how="left", op='intersects')
    intersections_with_traffic_lights=nodes_in_500m.loc[nodes_in_500m['crossing']=='traffic_signals']
    intersections_without_traffic_lights=nodes_in_500m.loc[(nodes_in_500m['crossing']!='traffic_signals')&(nodes_in_500m['crossing']!='')]
    
    # drop null rows in panoId column
    intersections_with_traffic_lights=intersections_with_traffic_lights.dropna(subset=['panoId'])
    intersections_without_traffic_lights=intersections_without_traffic_lights.dropna(subset=['panoId'])
    # group by panoId to get counts of points within their buffers
    intersections_with_traffic_lights_count=intersections_with_traffic_lights.groupby(['panoId']).size().reset_index(name='counts')
    intersections_without_traffic_lights_count=intersections_without_traffic_lights.groupby(['panoId']).size().reset_index(name='counts')
    # left join the count to the img_location_df
    intersections_with_traffic_lights_count_left_joined=pd.merge(img_location_df[['panoId']],
                                                                intersections_with_traffic_lights_count,
                                                                on='panoId',
                                                                how='left'
                                                                )
    intersections_without_traffic_lights_count_left_joined=pd.merge(img_location_df[['panoId']],
                                                                intersections_without_traffic_lights_count,
                                                                on='panoId',
                                                                how='left'
                                                                )
    # fill na with 0
    intersections_with_traffic_lights_count_left_joined=intersections_with_traffic_lights_count_left_joined.fillna(0)
    intersections_without_traffic_lights_count_left_joined=intersections_without_traffic_lights_count_left_joined.fillna(0)
    # add 'pano=' to the panoId coulumn
    intersections_with_traffic_lights_count_left_joined['panoId']='pano='+intersections_with_traffic_lights_count_left_joined['panoId']
    intersections_without_traffic_lights_count_left_joined['panoId']='pano='+intersections_without_traffic_lights_count_left_joined['panoId']
    # save it to tabular_data folder
    print(intersections_with_traffic_lights_count_left_joined)
    print(intersections_without_traffic_lights_count_left_joined)
    intersections_with_traffic_lights_count_left_joined.to_csv(os.path.join(root,'data/tabular_data/{}/intersections_with_traffic_lights.csv'.format(city)))
    intersections_without_traffic_lights_count_left_joined.to_csv(os.path.join(root,'data/tabular_data/{}/intersections_without_traffic_lights.csv'.format(city)))

#### Number of cul-de-sac / dead-end

##### get crossing with and without traffic signals from highway nodes in OSM

In [ ]:
city_list=[
    'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# parameter for osmnx
tags={'highway':['turning_circle','turning_loop']}
# go through cities in the list
for city in tqdm.tqdm(city_list):
    cul_de_sac=ox.geometries_from_place(city, tags)
    cul_de_sac.to_file(os.path.join(root,'data/network/{}_street_network/{}_cul_de_sac.shp').format(city,city))

In [ ]:
# Number of cul-de-sac / dead-end
city_list=[
    'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

for city in city_list:
    # get the remaining images' locations
    img_location=os.path.join(root,'data/meta_data_{}/{}_remaining_points.csv'.format(city,city))
    #load the csv file and convert it to gdf
    img_location_df=pd.read_csv(img_location)
    img_location_df['geometry']=img_location_df.apply(lambda x: Point((float(x.panoLon), float(x.panoLat))), axis=1)
    img_location_gdf=gpd.GeoDataFrame(img_location_df, geometry='geometry')
    img_location_gdf=img_location_gdf.set_crs(epsg=4326)
    #convert to Pseudo-Mercator
    img_location_gdf=img_location_gdf.to_crs("EPSG:3857")
    
    # get the edge file from network folder
    cul_de_sac=os.path.join(root,'data/network/{}_street_network/{}_cul_de_sac.shp'.format(city,city))
    cul_de_sac_gdf=gpd.read_file(cul_de_sac)
    # convert to Pseudo-Mercator
    cul_de_sac_gdf=cul_de_sac_gdf.to_crs("EPSG:3857")
    print(cul_de_sac_gdf)
    
    # create 500m buffer around img_location_gdf
    buffer_500m = img_location_gdf.copy()
    buffer_500m["geometry"]=img_location_gdf.geometry.buffer(500)
    # coduct spatial join to extract points that intersect with the 500m buffer
    cul_de_sac_500m = gpd.sjoin(cul_de_sac_gdf, buffer_500m, how="left", op='intersects')
    print(cul_de_sac)
    
    # drop null rows in panoId column
    cul_de_sac_500m=cul_de_sac_500m.dropna(subset=['panoId'])
    # group by panoId to get counts of points within their buffers
    cul_de_sac_500m_count=cul_de_sac_500m.groupby(['panoId']).size().reset_index(name='counts')
    # left join the count to the img_location_df
    cul_de_sac_500m_count_left_joined=pd.merge(img_location_df[['panoId']],
                                               cul_de_sac_500m_count,
                                               on='panoId',
                                               how='left'
                                              )
    # fill na with 0
    cul_de_sac_500m_count_left_joined=cul_de_sac_500m_count_left_joined.fillna(0)
    # add 'pano=' to the panoId coulumn
    cul_de_sac_500m_count_left_joined['panoId']='pano='+cul_de_sac_500m_count_left_joined['panoId']
    # save it to tabular_data folder
    print(cul_de_sac_500m_count_left_joined)
    cul_de_sac_500m_count_left_joined.to_csv(os.path.join(root,'data/tabular_data/{}/cul_de_sac.csv'.format(city)))

#### Number of points of interest

##### get amenity of points in OSM

In [ ]:
city_list=[
    'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# parameter for osmnx
tags={'amenity':True}

# go through cities in the list
for city in tqdm.tqdm(city_list):
    # get pois
    pois=ox.geometries_from_place(city, tags)
    # only select point features and drop 'nodes' column
    pois=pois.loc[pois['element_type']=='node']
    pois=pois.drop(['nodes'],axis=1)
    pois=pois[['unique_id','osmid','name','amenity','geometry']]
    pois.to_file(os.path.join(root,'data/network/{}_street_network/{}_poi.shp').format(city,city))

In [ ]:
# Number of pois
city_list=[
    'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

for city in city_list:
    # get the remaining images' locations
    img_location=os.path.join(root,'data/meta_data_{}/{}_remaining_points.csv'.format(city,city))
    #load the csv file and convert it to gdf
    img_location_df=pd.read_csv(img_location)
    img_location_df['geometry']=img_location_df.apply(lambda x: Point((float(x.panoLon), float(x.panoLat))), axis=1)
    img_location_gdf=gpd.GeoDataFrame(img_location_df, geometry='geometry')
    img_location_gdf=img_location_gdf.set_crs(epsg=4326)
    #convert to Pseudo-Mercator
    img_location_gdf=img_location_gdf.to_crs("EPSG:3857")
    
    # get the poi file from network folder
    pois=os.path.join(root,'data/network/{}_street_network/{}_poi.shp'.format(city,city))
    pois_gdf=gpd.read_file(pois)
    # convert to Pseudo-Mercator
    pois_gdf=pois_gdf.to_crs("EPSG:3857")
    print(pois_gdf)
    
    # create 500m buffer around img_location_gdf
    buffer_500m = img_location_gdf.copy()
    buffer_500m["geometry"]=img_location_gdf.geometry.buffer(500)
    # coduct spatial join to extract points that intersect with the 500m buffer
    pois_in_500m = gpd.sjoin(pois_gdf, buffer_500m, how="left", op='intersects')
    print(pois_in_500m)
    
    # drop null rows in panoId column
    pois_in_500m=pois_in_500m.dropna(subset=['panoId'])
    # group by panoId to get counts of points within their buffers
    pois_in_500m_count=pois_in_500m.groupby(['panoId']).size().reset_index(name='counts')
    # left join the count to the img_location_df
    pois_in_500m_count_left_joined=pd.merge(img_location_df[['panoId']],
                                            pois_in_500m_count,
                                            on='panoId',
                                            how='left'
                                           )
    # fill na with 0
    pois_in_500m_count_left_joined=pois_in_500m_count_left_joined.fillna(0)
    # add 'pano=' to the panoId coulumn
    pois_in_500m_count_left_joined['panoId']='pano='+pois_in_500m_count_left_joined['panoId']
    # save it to tabular_data folder
    print(pois_in_500m_count_left_joined)
    pois_in_500m_count_left_joined.to_csv(os.path.join(root,'data/tabular_data/{}/poi.csv'.format(city)))

#### Number of transit facilities

##### get MRT (Train) stations from OSM

In [ ]:
city_list=[
    'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# parameter for osmnx
tags={'railway':'stop'}

# go through cities in the list
for city in tqdm.tqdm(city_list):
    # get transit
    transit=ox.geometries_from_place(city, tags)
    
    # drop duplicates
    transit=transit.drop_duplicates(subset=['name'])
    print(transit)
    transit.to_file(os.path.join(root,'data/network/{}_street_network/{}_transit.shp').format(city,city))

In [ ]:
# Number of transit facilities
city_list=[
    'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

for city in city_list:
    # get the remaining images' locations
    img_location=os.path.join(root,'data/meta_data_{}/{}_remaining_points.csv'.format(city,city))
    #load the csv file and convert it to gdf
    img_location_df=pd.read_csv(img_location)
    img_location_df['geometry']=img_location_df.apply(lambda x: Point((float(x.panoLon), float(x.panoLat))), axis=1)
    img_location_gdf=gpd.GeoDataFrame(img_location_df, geometry='geometry')
    img_location_gdf=img_location_gdf.set_crs(epsg=4326)
    #convert to Pseudo-Mercator
    img_location_gdf=img_location_gdf.to_crs("EPSG:3857")
    
    # get the transit file from network folder
    transit=os.path.join(root,'data/network/{}_street_network/{}_transit.shp'.format(city,city))
    transit_gdf=gpd.read_file(transit)
    # convert to Pseudo-Mercator
    transit_gdf=transit_gdf.to_crs("EPSG:3857")
    print(transit_gdf)
    
    # create 500m buffer around img_location_gdf
    buffer_500m = img_location_gdf.copy()
    buffer_500m["geometry"]=img_location_gdf.geometry.buffer(500)
    # coduct spatial join to extract points that intersect with the 500m buffer
    transit_in_500m = gpd.sjoin(transit_gdf, buffer_500m, how="left", op='intersects')
    print(pois_in_500m)
    
    # drop null rows in panoId column
    transit_in_500m=transit_in_500m.dropna(subset=['panoId'])
    # group by panoId to get counts of points within their buffers
    transit_in_500m_count=transit_in_500m.groupby(['panoId']).size().reset_index(name='counts')
    # left join the count to the img_location_df
    transit_in_500m_count_left_joined=pd.merge(img_location_df[['panoId']],
                                            transit_in_500m_count,
                                            on='panoId',
                                            how='left'
                                           )
    # fill na with 0
    transit_in_500m_count_left_joined=transit_in_500m_count_left_joined.fillna(0)
    # add 'pano=' to the panoId coulumn
    transit_in_500m_count_left_joined['panoId']='pano='+transit_in_500m_count_left_joined['panoId']
    # save it to tabular_data folder
    print(transit_in_500m_count_left_joined)
    transit_in_500m_count_left_joined.to_csv(os.path.join(root,'data/tabular_data/{}/transit.csv'.format(city)))

### 100m buffer
- Type of road
- Type of pavement (paved vs unpaved)
- Road width
- Number of traffic lanes
- Presence of off-street parking lot spaces
- Number of speed bumps / choker / roundabout
- Vehicular speed limit

#### Type of road

In [ ]:
# Number of type of road
city_list=[
    'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

for city in city_list:
    # get the remaining images' locations
    img_location=os.path.join(root,'data/meta_data_{}/{}_remaining_points.csv'.format(city,city))
    #load the csv file and convert it to gdf
    img_location_df=pd.read_csv(img_location)
    img_location_df['geometry']=img_location_df.apply(lambda x: Point((float(x.panoLon), float(x.panoLat))), axis=1)
    img_location_gdf=gpd.GeoDataFrame(img_location_df, geometry='geometry')
    img_location_gdf=img_location_gdf.set_crs(epsg=4326)
    #convert to Pseudo-Mercator
    img_location_gdf=img_location_gdf.to_crs("EPSG:3857")
    
    # get the transit file from network folder
    road=os.path.join(root,'data/network/{}_street_network/edges.shp'.format(city))
    road_gdf=gpd.read_file(road)
    # convert to Pseudo-Mercator
    road_gdf=road_gdf.to_crs("EPSG:3857")
    
    # create 100m buffer around img_location_gdf
    buffer_100m = img_location_gdf.copy()
    buffer_100m["geometry"]=img_location_gdf.geometry.buffer(100)
    # coduct spatial join to extract points that intersect with the 100m buffer
    road_in_100m = gpd.sjoin(road_gdf, buffer_100m, how="left", op='intersects')
    
    # drop null rows in panoId column
    road_in_100m=road_in_100m.dropna(subset=['panoId'])
    # calculate the lengh of each line feature
    road_in_100m['length']=road_in_100m['geometry'].length
    # convert type of road ('highway' column) to numeric values
    non_others_list=['service','track','primary','primary_link',
                     'secondary','secondary_link','road','tertiary',
                     'tertiary_link','residential','living_street','pedestrian',
                     'footway','path','cycleway'
                    ]
    cond_list=[(road_in_100m['highway']=='service')|(road_in_100m['highway']=='track'),
               (road_in_100m['highway']=='primary')|(road_in_100m['highway']=='primary_link'),
               (road_in_100m['highway']=='secondary')|(road_in_100m['highway']=='secondary_link'),
               road_in_100m['highway']=='road',
               (road_in_100m['highway']=='tertiary')|(road_in_100m['highway']=='tertiary_link'),
               (road_in_100m['highway']=='residential')|(road_in_100m['highway']=='living_street')|\
               (road_in_100m['highway']=='pedestrian')|(road_in_100m['highway']=='footway')|\
               (road_in_100m['highway']=='path'),
               road_in_100m['highway']=='cycleway',
               ~road_in_100m['highway'].isin(non_others_list)
              ]
    choice_list=[0.1,0.2,0.4,0.5,0.6,0.8,1,0]
    road_in_100m['road_type_num'] = np.select(cond_list, choice_list)
    road_in_100m['road_type_score']=road_in_100m['road_type_num']*road_in_100m['length']
    print(road_in_100m)
    # group by panoId to get sum
    road_in_100m_grouped=road_in_100m.groupby(['panoId']).agg({'length':'sum','road_type_score':'sum'}).reset_index()
    road_in_100m_grouped['road_type_mean']=road_in_100m_grouped['road_type_score']/road_in_100m_grouped['length']
    print(road_in_100m_grouped)
    # left join the count to the img_location_df
    road_in_100m_grouped_left_joined=pd.merge(img_location_df[['panoId']],
                                            road_in_100m_grouped,
                                            on='panoId',
                                            how='left'
                                           )
    # fill na with 0
    road_in_100m_grouped_left_joined=road_in_100m_grouped_left_joined.fillna(0)
    # add 'pano=' to the panoId coulumn
    road_in_100m_grouped_left_joined['panoId']='pano='+road_in_100m_grouped_left_joined['panoId']
    # save it to tabular_data folder
    print(road_in_100m_grouped_left_joined)
    road_in_100m_grouped_left_joined.to_csv(os.path.join(root,'data/tabular_data/{}/road_type.csv'.format(city)))

#### Type of pavement (paved vs unpaved)

##### get surface from OSM

In [ ]:
city_list=[
    'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# parameter for osmnx
tags={'surface':True}

# go through cities in the list
for city in tqdm.tqdm(city_list):
    # get surface
    surface=ox.geometries_from_place(city, tags)
    # only select line features and drop 'nodes' column
    surface=surface.loc[surface['element_type']=='way']
    surface=surface.drop(['nodes'],axis=1)
    # keep only unique_id, osmid, surface, geometry
    surface=surface[['unique_id','osmid','surface','geometry']]
    print(surface)
    surface.to_file(os.path.join(root,'data/network/{}_street_network/{}_surface.shp').format(city,city))

In [ ]:
# type of pavement
city_list=[
    'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

for city in city_list:
    # get the remaining images' locations
    img_location=os.path.join(root,'data/meta_data_{}/{}_remaining_points.csv'.format(city,city))
    #load the csv file and convert it to gdf
    img_location_df=pd.read_csv(img_location)
    img_location_df['geometry']=img_location_df.apply(lambda x: Point((float(x.panoLon), float(x.panoLat))), axis=1)
    img_location_gdf=gpd.GeoDataFrame(img_location_df, geometry='geometry')
    img_location_gdf=img_location_gdf.set_crs(epsg=4326)
    #convert to Pseudo-Mercator
    img_location_gdf=img_location_gdf.to_crs("EPSG:3857")
    
    # get the surface file from network folder
    road=os.path.join(root,'data/network/{}_street_network/{}_surface.shp'.format(city,city))
    road_gdf=gpd.read_file(road)
    # convert to Pseudo-Mercator
    road_gdf=road_gdf.to_crs("EPSG:3857")
    
    # create 100m buffer around img_location_gdf
    buffer_100m = img_location_gdf.copy()
    buffer_100m["geometry"]=img_location_gdf.geometry.buffer(100)
    # coduct spatial join to extract points that intersect with the 100m buffer
    road_in_100m = gpd.sjoin(road_gdf, buffer_100m, how="left", op='intersects')
    
    # drop null rows in panoId column
    road_in_100m=road_in_100m.dropna(subset=['panoId'])
    # calculate the lengh of each line feature
    road_in_100m['length']=road_in_100m['geometry'].length
    # convert type of pavement ('surface' column) to numeric values
    non_others_list=['unhewn_cobblestone','cobblestone','sett','metal',
                     'wood','paved','concrete:lanes','concrete:plates',
                     'paving_stones','asphalt','concrete'
                    ]
    cond_list=[(road_in_100m['surface']=='unhewn_cobblestone')|(road_in_100m['surface']=='cobblestone'),
               (road_in_100m['surface']=='sett')|(road_in_100m['surface']=='metal')|(road_in_100m['surface']=='wood'),
               road_in_100m['surface']=='paved',
               (road_in_100m['surface']=='concrete:lanes')|(road_in_100m['surface']=='concrete:plates')|(road_in_100m['surface']=='paving_stones'),
               (road_in_100m['surface']=='asphalt')|(road_in_100m['surface']=='concrete'),
               ~road_in_100m['surface'].isin(non_others_list)
              ]
    choice_list=[0.2,0.4,0.5,0.6,1,0]
    road_in_100m['surface_type_num'] = np.select(cond_list, choice_list)
    road_in_100m['surface_type_score']=road_in_100m['surface_type_num']*road_in_100m['length']
    print(road_in_100m)
    # group by panoId to get sum
    road_in_100m_grouped=road_in_100m.groupby(['panoId']).agg({'length':'sum','surface_type_score':'sum'}).reset_index()
    road_in_100m_grouped['surface_type_mean']=road_in_100m_grouped['surface_type_score']/road_in_100m_grouped['length']
    print(road_in_100m_grouped)
    # left join the count to the img_location_df
    road_in_100m_grouped_left_joined=pd.merge(img_location_df[['panoId']],
                                            road_in_100m_grouped,
                                            on='panoId',
                                            how='left'
                                           )
    # fill na with 0
    road_in_100m_grouped_left_joined=road_in_100m_grouped_left_joined.fillna(0)
    # add 'pano=' to the panoId coulumn
    road_in_100m_grouped_left_joined['panoId']='pano='+road_in_100m_grouped_left_joined['panoId']
    # save it to tabular_data folder
    print(road_in_100m_grouped_left_joined)
    road_in_100m_grouped_left_joined.to_csv(os.path.join(root,'data/tabular_data/{}/surface_type.csv'.format(city)))

#### Road width

In [ ]:
# Road width
city_list=[
    'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

for city in city_list:
    # get the remaining images' locations
    img_location=os.path.join(root,'data/meta_data_{}/{}_remaining_points.csv'.format(city,city))
    #load the csv file and convert it to gdf
    img_location_df=pd.read_csv(img_location)
    img_location_df['geometry']=img_location_df.apply(lambda x: Point((float(x.panoLon), float(x.panoLat))), axis=1)
    img_location_gdf=gpd.GeoDataFrame(img_location_df, geometry='geometry')
    img_location_gdf=img_location_gdf.set_crs(epsg=4326)
    #convert to Pseudo-Mercator
    img_location_gdf=img_location_gdf.to_crs("EPSG:3857")
    
    # get the width file from network folder
    road=os.path.join(root,'data/network/{}_street_network/edges.shp'.format(city))
    road_gdf=gpd.read_file(road)
    # convert to Pseudo-Mercator
    road_gdf=road_gdf.to_crs("EPSG:3857")
    # drop lines without width info
    road_gdf=road_gdf.dropna(subset=['width'])
    road_gdf=road_gdf.loc[road_gdf['width'].str.isdigit()]
    road_gdf['width']=road_gdf['width'].astype(int)
    print(road_gdf)
    
    # create 100m buffer around img_location_gdf
    buffer_100m = img_location_gdf.copy()
    buffer_100m["geometry"]=img_location_gdf.geometry.buffer(100)
    # coduct spatial join to extract points that intersect with the 100m buffer
    road_in_100m = gpd.sjoin(road_gdf, buffer_100m, how="left", op='intersects')
    
    # drop null rows in panoId column
    road_in_100m=road_in_100m.dropna(subset=['panoId'])
    # calculate the lengh of each line feature
    road_in_100m['length']=road_in_100m['geometry'].length
    # convert road width ('width' column) to numeric values
    cond_list=[road_in_100m['width']<=10,
               road_in_100m['width']>10
              ]
    choice_list=[road_in_100m['width']/10,1]
    road_in_100m['road_width_num'] = np.select(cond_list, choice_list)
    road_in_100m['road_width_score']=road_in_100m['road_width_num']*road_in_100m['length']
    print(road_in_100m)
    # group by panoId to get sum
    road_in_100m_grouped=road_in_100m.groupby(['panoId']).agg({'length':'sum','road_width_score':'sum'}).reset_index()
    road_in_100m_grouped['road_width_mean']=road_in_100m_grouped['road_width_score']/road_in_100m_grouped['length']
    print(road_in_100m_grouped)
    # left join the count to the img_location_df
    road_in_100m_grouped_left_joined=pd.merge(img_location_df[['panoId']],
                                            road_in_100m_grouped,
                                            on='panoId',
                                            how='left'
                                           )
    # fill na with 0
    road_in_100m_grouped_left_joined=road_in_100m_grouped_left_joined.fillna(0)
    # add 'pano=' to the panoId coulumn
    road_in_100m_grouped_left_joined['panoId']='pano='+road_in_100m_grouped_left_joined['panoId']
    # save it to tabular_data folder
    print(road_in_100m_grouped_left_joined)
    road_in_100m_grouped_left_joined.to_csv(os.path.join(root,'data/tabular_data/{}/road_width.csv'.format(city)))

#### Number of traffic lanes

In [ ]:
# traffic lanes
city_list=[
    'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

for city in tqdm.tqdm(city_list):
    # get the remaining images' locations
    img_location=os.path.join(root,'data/meta_data_{}/{}_remaining_points.csv'.format(city,city))
    #load the csv file and convert it to gdf
    img_location_df=pd.read_csv(img_location)
    img_location_df['geometry']=img_location_df.apply(lambda x: Point((float(x.panoLon), float(x.panoLat))), axis=1)
    img_location_gdf=gpd.GeoDataFrame(img_location_df, geometry='geometry')
    img_location_gdf=img_location_gdf.set_crs(epsg=4326)
    #convert to Pseudo-Mercator
    img_location_gdf=img_location_gdf.to_crs("EPSG:3857")
    
    # get the road file from network folder
    road=os.path.join(root,'data/network/{}_street_network/edges.shp'.format(city))
    road_gdf=gpd.read_file(road)
    # convert to Pseudo-Mercator
    road_gdf=road_gdf.to_crs("EPSG:3857")
    # drop na
    road_gdf['lanes']=road_gdf['lanes'].fillna('0')
    # drop cycle way
    road_gdf=road_gdf.loc[road_gdf['highway']!='cycleway']
    road_gdf=road_gdf.loc[road_gdf['lanes'].str.isdigit()]
    road_gdf['lanes']=road_gdf['lanes'].astype(int)
    
    # create 100m buffer around img_location_gdf
    buffer_100m = img_location_gdf.copy()
    buffer_100m["geometry"]=img_location_gdf.geometry.buffer(100)
    # coduct spatial join to extract points that intersect with the 100m buffer
    road_in_100m = gpd.sjoin(road_gdf, buffer_100m, how="left", op='intersects')
    
    # drop null rows in panoId column
    road_in_100m=road_in_100m.dropna(subset=['panoId'])
    # calculate the lengh of each line feature
    road_in_100m['length']=road_in_100m['geometry'].length
    # convert road lanes ('lanes' column) to numeric values
    cond_list=[(road_in_100m['lanes']==1)|(road_in_100m['lanes']==0),
               road_in_100m['lanes']==2,
               road_in_100m['lanes']==3,
               road_in_100m['lanes']==4,
               road_in_100m['lanes']>=5
              ]
    choice_list=[1,0.75,0.5,0.25,0]
    road_in_100m['traffic_lane_num'] = np.select(cond_list, choice_list)
    road_in_100m['traffic_lane_score']=road_in_100m['traffic_lane_num']*road_in_100m['length']
    print(road_in_100m)
    # group by panoId to get sum
    road_in_100m_grouped=road_in_100m.groupby(['panoId']).agg({'length':'sum','traffic_lane_score':'sum'}).reset_index()
    road_in_100m_grouped['traffic_lane_mean']=road_in_100m_grouped['traffic_lane_score']/road_in_100m_grouped['length']
    print(road_in_100m_grouped)
    # left join the count to the img_location_df
    road_in_100m_grouped_left_joined=pd.merge(img_location_df[['panoId']],
                                            road_in_100m_grouped,
                                            on='panoId',
                                            how='left'
                                           )
    # fill na with 0
    road_in_100m_grouped_left_joined=road_in_100m_grouped_left_joined.fillna(0)
    # add 'pano=' to the panoId coulumn
    road_in_100m_grouped_left_joined['panoId']='pano='+road_in_100m_grouped_left_joined['panoId']
    # save it to tabular_data folder
    print(road_in_100m_grouped_left_joined)
    road_in_100m_grouped_left_joined.to_csv(os.path.join(root,'data/tabular_data/{}/traffic_lane.csv'.format(city)))

#### Presence of off-street parking lot spaces

##### get highway from OSM

In [ ]:
city_list=[
    'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# parameter for osmnx
tags={'parking:lane:left':True,'parking:lane:right':True,'parking:lane:both':True}

# go through cities in the list
for city in tqdm.tqdm(city_list):
    # get street_parking
    street_parking=ox.geometries_from_place(city, tags)
    
    # only select line features and drop 'nodes' column
    street_parking=street_parking.loc[street_parking['element_type']=='way']
    street_parking=street_parking.drop(['nodes'],axis=1)
    col_keep=['unique_id','osmid','name','parking:lane:left','parking:lane:right','parking:lane:both','geometry']
    try:
        street_parking=street_parking[col_keep]
    except Exception as error:
        missing_col=str(error).split(']')[0].replace('"[','').replace("'",'')
        print(missing_col)
        col_keep.remove(missing_col)
        street_parking=street_parking[col_keep]
    # only select parallel, diagonal, perpendicular, marked in parking:lane cols
    parking_cols=[col for col in street_parking.columns if col.startswith('parking:lane')]
    street_parking['total_score']=0
    for parking_col in parking_cols:
        street_parking['score_'+parking_col]=0
        street_parking.loc[(street_parking[parking_col]=='parallel')|\
                           (street_parking[parking_col]=='diagonal')|\
                           (street_parking[parking_col]=='perpendicular')|\
                           (street_parking[parking_col]=='marked'),
                            ['score_'+parking_col]]=1
        street_parking['total_score']=street_parking['total_score']+street_parking['score_'+parking_col]
    street_parking=street_parking.loc[street_parking['total_score']>0]
    print(street_parking)
    street_parking.to_file(os.path.join(root,'data/network/{}_street_network/{}_street_parking.shp').format(city,city))

In [46]:
# off street parking
city_list=[
    'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

for city in city_list:
    # get the remaining images' locations
    img_location=os.path.join(root,'data/meta_data_{}/{}_remaining_points.csv'.format(city,city))
    #load the csv file and convert it to gdf
    img_location_df=pd.read_csv(img_location)
    img_location_df['geometry']=img_location_df.apply(lambda x: Point((float(x.panoLon), float(x.panoLat))), axis=1)
    img_location_gdf=gpd.GeoDataFrame(img_location_df, geometry='geometry')
    img_location_gdf=img_location_gdf.set_crs(epsg=4326)
    #convert to Pseudo-Mercator
    img_location_gdf=img_location_gdf.to_crs("EPSG:3857")
    
    # get the road file from network folder
    road=os.path.join(root,'data/network/{}_street_network/edges.shp'.format(city))
    road_gdf=gpd.read_file(road)
    # convert to Pseudo-Mercator
    road_gdf=road_gdf.to_crs("EPSG:3857")
    
    # load street parking data
    street_parking=gpd.read_file(os.path.join(root,'data/network/{}_street_network/{}_street_parking.shp').format(city,city))
    street_parking['osmid']=street_parking['osmid'].astype(str)
    
    # vertically split osmid with ',' for road_gdf
    road_gdf=road_gdf.assign(osmid=road_gdf['osmid'].str.split(',')).explode('osmid')
    road_gdf['osmid']=road_gdf['osmid'].str.replace("\\[","").str.replace("\\]","")
    
    # join street parking to road by osmid
    road_gdf=road_gdf.merge(street_parking,
                       on='osmid',
                       how='left'
                      )
    # convert road_gdf back to gdf again
    road_gdf = gpd.GeoDataFrame(road_gdf, crs="EPSG:3857", geometry='geometry_x')
    
    # create 100m buffer around img_location_gdf
    buffer_100m = img_location_gdf.copy()
    buffer_100m["geometry"]=img_location_gdf.geometry.buffer(100)
    # coduct spatial join to extract points that intersect with the 100m buffer
    road_in_100m = gpd.sjoin(road_gdf, buffer_100m, how="left", op='intersects')
    
    # drop null rows in panoId column
    road_in_100m=road_in_100m.dropna(subset=['panoId'])
    # calculate the lengh of each line feature
    road_in_100m['length']=road_in_100m['geometry_x'].length
    # convert street parking to numeric values
    road_in_100m['total_scor']=road_in_100m['total_scor'].fillna(0)
    cond_list=[road_in_100m['total_scor']>0,
               road_in_100m['total_scor']==0]
    choice_list=[0,1]
    road_in_100m['street_parking_num'] = np.select(cond_list, choice_list)
    road_in_100m['street_parking_score']=road_in_100m['street_parking_num']*road_in_100m['length']
    print(road_in_100m)
    # group by panoId to get sum
    road_in_100m_grouped=road_in_100m.groupby(['panoId']).agg({'length':'sum','street_parking_score':'sum'}).reset_index()
    road_in_100m_grouped['street_parking_mean']=road_in_100m_grouped['street_parking_score']/road_in_100m_grouped['length']
    print(road_in_100m_grouped)
    # left join the count to the img_location_df
    road_in_100m_grouped_left_joined=pd.merge(img_location_df[['panoId']],
                                            road_in_100m_grouped,
                                            on='panoId',
                                            how='left'
                                           )
    # fill na with 0
    road_in_100m_grouped_left_joined=road_in_100m_grouped_left_joined.fillna(0)
    # add 'pano=' to the panoId coulumn
    road_in_100m_grouped_left_joined['panoId']='pano='+road_in_100m_grouped_left_joined['panoId']
    # save it to tabular_data folder
    print(road_in_100m_grouped_left_joined)
    road_in_100m_grouped_left_joined.to_csv(os.path.join(root,'data/tabular_data/{}/street_parking.csv'.format(city)))

/home/ec2-user/anaconda3/envs/mxnet_latest_p37/lib/python3.7/site-packages/ipykernel/__main__.py:31: FutureWarning: The default value of regex will change from True to False in a future version.


                 u           v  key       osmid oneway lanes   ref  \
0         25451929  6749812859    0    49961799   True     5   ECP   
0         25451929  6749812859    0    49961799   True     5   ECP   
0         25451929  6749812859    0    49961799   True     5   ECP   
0         25451929  6749812859    0    49961799   True     5   ECP   
1         25451929  6749812859    0   718881456   True     5   ECP   
...            ...         ...  ...         ...    ...   ...   ...   
313998  8336385420  8336385421    0   483606240   True  None  None   
314007  8430404643  8430404652    0   654997378   True     3  None   
314007  8430404643  8430404652    0   654997378   True     3  None   
314008  8430404643  8430404652    0   821527756   True     3  None   
314008  8430404643  8430404652    0   821527756   True     3  None   

                      name_x   highway maxspeed  ...  score_pa_2 geometry_y  \
0         East Coast Parkway  motorway       70  ...         NaN       None   
0

KeyboardInterrupt: 

#### Number of speed bumps / choker / roundabout

##### get traffic_calming from OSM

In [ ]:
city_list=[
    'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# parameter for osmnx
tags={'traffic_calming':True}

# go through cities in the list
for city in tqdm.tqdm(city_list):
    # get traffic_calming
    traffic_calming=ox.geometries_from_place(city, tags)
    # only select line features and drop 'nodes' column
    traffic_calming=traffic_calming.loc[traffic_calming['element_type']=='node']
    traffic_calming=traffic_calming.drop(['nodes'],axis=1)
    # keep only unique_id, osmid, traffic_calming, geometry
    traffic_calming=traffic_calming[['unique_id','osmid','traffic_calming','geometry']]
    print(traffic_calming)
    traffic_calming.to_file(os.path.join(root,'data/network/{}_street_network/{}_traffic_calming.shp').format(city,city))

In [34]:
# Number of speed bumps / choker / roundabout
city_list=[
    'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

for city in city_list:
    # get the remaining images' locations
    img_location=os.path.join(root,'data/meta_data_{}/{}_remaining_points.csv'.format(city,city))
    #load the csv file and convert it to gdf
    img_location_df=pd.read_csv(img_location)
    img_location_df['geometry']=img_location_df.apply(lambda x: Point((float(x.panoLon), float(x.panoLat))), axis=1)
    img_location_gdf=gpd.GeoDataFrame(img_location_df, geometry='geometry')
    img_location_gdf=img_location_gdf.set_crs(epsg=4326)
    #convert to Pseudo-Mercator
    img_location_gdf=img_location_gdf.to_crs("EPSG:3857")
    
    # get the road file from network folder
    road=os.path.join(root,'data/network/{}_street_network/{}_traffic_calming.shp'.format(city,city))
    road_gdf=gpd.read_file(road)
    # convert to Pseudo-Mercator
    road_gdf=road_gdf.to_crs("EPSG:3857")
    
    # create 100m buffer around img_location_gdf
    buffer_100m = img_location_gdf.copy()
    buffer_100m["geometry"]=img_location_gdf.geometry.buffer(100)
    # coduct spatial join to extract points that intersect with the 100m buffer
    road_in_100m = gpd.sjoin(road_gdf, buffer_100m, how="left", op='intersects')
    
    # drop null rows in panoId column
    road_in_100m=road_in_100m.dropna(subset=['panoId'])
    
    # 1 to all the rows
    road_in_100m['traffic_calming']=1
    # group by panoId to get sum
    road_in_100m_grouped=road_in_100m.groupby(['panoId']).agg({'traffic_calming':'sum'}).reset_index()
    road_in_100m_grouped['traffic_calming']=road_in_100m_grouped['traffic_calming']
    # left join the count to the img_location_df
    road_in_100m_grouped_left_joined=pd.merge(img_location_df[['panoId']],
                                            road_in_100m_grouped,
                                            on='panoId',
                                            how='left'
                                           )
    # fill na with 0
    road_in_100m_grouped_left_joined=road_in_100m_grouped_left_joined.fillna(0)
    # add 'pano=' to the panoId coulumn
    road_in_100m_grouped_left_joined['panoId']='pano='+road_in_100m_grouped_left_joined['panoId']
    # save it to tabular_data folder
    print(road_in_100m_grouped_left_joined)
    road_in_100m_grouped_left_joined.to_csv(os.path.join(root,'data/tabular_data/{}/traffic_calming.csv'.format(city)))

                                                 panoId  traffic_calming
0                           pano=EIzSniBOXDgT4m2fGH2SNQ              0.0
1                           pano=tzvazBY0ag5RgAsvJgHvZA              0.0
2                           pano=FJjCz3jIAWYYnGq4PvR9HA              0.0
3                           pano=TfQN73pv-6mAZy2QnMIAPA              0.0
4                           pano=PATYQYa4O43_LjJ_JMbrFw              0.0
...                                                 ...              ...
5828  pano=CAoSLEFGMVFpcE9YcGFGMVpBOURjN21NS0NQS1h3L...              0.0
5829                        pano=4I7X92FmvGD7ALNXMZQuAg              0.0
5830                        pano=EghtrEJcRO-l7XlA21zaNQ              0.0
5831                        pano=O6UhfdrDWVrKDmJTaOLmFA              0.0
5832                        pano=uvufId4xvxyRl-4TUehx_A              0.0

[5833 rows x 2 columns]
                           panoId  traffic_calming
0     pano=d1hU7t4QKuXqckKk29ENTQ              0

#### Vehicular speed limit

In [ ]:
# speed limit
city_list=[
    'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

for city in city_list:
    # get the remaining images' locations
    img_location=os.path.join(root,'data/meta_data_{}/{}_remaining_points.csv'.format(city,city))
    #load the csv file and convert it to gdf
    img_location_df=pd.read_csv(img_location)
    img_location_df['geometry']=img_location_df.apply(lambda x: Point((float(x.panoLon), float(x.panoLat))), axis=1)
    img_location_gdf=gpd.GeoDataFrame(img_location_df, geometry='geometry')
    img_location_gdf=img_location_gdf.set_crs(epsg=4326)
    #convert to Pseudo-Mercator
    img_location_gdf=img_location_gdf.to_crs("EPSG:3857")
    
    # get the road file from network folder
    road=os.path.join(root,'data/network/{}_street_network/edges.shp'.format(city))
    road_gdf=gpd.read_file(road)
    # convert to Pseudo-Mercator
    road_gdf=road_gdf.to_crs("EPSG:3857")
    road_gdf=road_gdf.dropna(subset=['maxspeed'])
    # drop cycle way
    road_gdf=road_gdf.loc[road_gdf['highway']!='cycleway']
    road_gdf=road_gdf.loc[road_gdf['maxspeed'].str.isdigit()]
    road_gdf['maxspeed']=road_gdf['maxspeed'].astype(int)
    
    # create 100m buffer around img_location_gdf
    buffer_100m = img_location_gdf.copy()
    buffer_100m["geometry"]=img_location_gdf.geometry.buffer(100)
    # coduct spatial join to extract points that intersect with the 100m buffer
    road_in_100m = gpd.sjoin(road_gdf, buffer_100m, how="left", op='intersects')
    
    # drop null rows in panoId column
    road_in_100m=road_in_100m.dropna(subset=['panoId'])
    # calculate the lengh of each line feature
    road_in_100m['length']=road_in_100m['geometry'].length
    # convert maxspeed to numeric values
    cond_list=[road_in_100m['maxspeed']<=20,
               (road_in_100m['maxspeed']>20)&(road_in_100m['maxspeed']<=40),
               (road_in_100m['maxspeed']>40)&(road_in_100m['maxspeed']<=60),
               (road_in_100m['maxspeed']>60)&(road_in_100m['maxspeed']<=80),
               road_in_100m['maxspeed']>=5
              ]
    choice_list=[1,0.75,0.5,0.25,0]
    road_in_100m['speed_limit_num'] = np.select(cond_list, choice_list)
    road_in_100m['speed_limit_score']=road_in_100m['speed_limit_num']*road_in_100m['length']
    print(road_in_100m)
    # group by panoId to get sum
    road_in_100m_grouped=road_in_100m.groupby(['panoId']).agg({'length':'sum','speed_limit_score':'sum'}).reset_index()
    road_in_100m_grouped['speed_limit_mean']=road_in_100m_grouped['speed_limit_score']/road_in_100m_grouped['length']
    print(road_in_100m_grouped)
    # left join the count to the img_location_df
    road_in_100m_grouped_left_joined=pd.merge(img_location_df[['panoId']],
                                            road_in_100m_grouped,
                                            on='panoId',
                                            how='left'
                                           )
    # fill na with 0
    road_in_100m_grouped_left_joined=road_in_100m_grouped_left_joined.fillna(1)
    # add 'pano=' to the panoId coulumn
    road_in_100m_grouped_left_joined['panoId']='pano='+road_in_100m_grouped_left_joined['panoId']
    # save it to tabular_data folder
    print(road_in_100m_grouped_left_joined)
    road_in_100m_grouped_left_joined.to_csv(os.path.join(root,'data/tabular_data/{}/speed_limit.csv'.format(city)))

## Land use
### 500m buffer

#### clean data

In [ ]:
# Singapore
root = "/home/ec2-user/SageMaker/"
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
df = gpd.read_file(os.path.join(root,'data/land_use/Singapore/G_MP19_LAND_USE_PL.kml'), driver='KML')
df['land_use']=df['Description'].str.split('<td>').str[1].str.split('</td>').str[0]

# convert land_use to residential, commercial, and industrial
df['land_use_modified']=None
cond_list=[(df['land_use']=='RESIDENTIAL')|\
           (df['land_use']=='COMMERCIAL & RESIDENTIAL')|\
           (df['land_use']=='RESIDENTIAL WITH COMMERCIAL AT 1ST STOREY')|\
           (df['land_use']=='RESIDENTIAL / INSTITUTION'),
           (df['land_use']=='COMMERCIAL')|\
           (df['land_use']=='HOTEL')|\
           (df['land_use']=='BUSINESS PARK')|\
           (df['land_use']=='COMMERCIAL / INSTITUTION')|\
           (df['land_use']=='BUSINESS PARK - WHITE'),
           (df['land_use']=='BUSINESS 2')|\
           (df['land_use']=='UTILITY')|\
           (df['land_use']=='BUSINESS 1')|\
           (df['land_use']=='PORT / AIRPORT')|\
           (df['land_use']=='BUSINESS 2 - WHITE')|\
           (df['land_use']=='BUSINESS 1 - WHITE')
          ]
choice_list=['residential','commercial','industrial']
df['land_use_modified'] = np.select(cond_list, choice_list)
df=df[['geometry','land_use_modified']]

# save as shp
df.to_file(os.path.join(root,'data/land_use/Singapore/land_use.shp'))

In [ ]:
# Tokyo
root = "/home/ec2-user/SageMaker/"
df=gpd.read_file(os.path.join(root,'data/land_use/Tokyo/A29-19_13000.shp'))

# convert A29_005 to residential, commercial, and industrial
df['land_use_modified']=None
cond_list=[(df['A29_005']=='第一種低層住居専用地域')|\
           (df['A29_005']=='第二種中高層住居専用地域')|\
           (df['A29_005']=='第一種中高層住居専用地域')|\
           (df['A29_005']=='第二種住居地域')|\
           (df['A29_005']=='第一種住居地域')|\
           (df['A29_005']=='準住居地域')|\
           (df['A29_005']=='第二種低層住居専用地域'),
           (df['A29_005']=='近隣商業地域')|\
           (df['A29_005']=='商業地域'),
           (df['A29_005']=='準工業地域')|\
           (df['A29_005']=='工業地域')|\
           (df['A29_005']=='工業専用地域')]
choice_list=['residential','commercial','industrial']
df['land_use_modified'] = np.select(cond_list, choice_list)
df=df[['geometry','land_use_modified']]

# save as shp
df.to_file(os.path.join(root,'data/land_use/Tokyo/land_use.shp'))

#### extract land use mix

In [ ]:
city_list=[
    'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# go through cities in the list
for city in city_list:
    # location data
    location=os.path.join(root,'data/meta_data_{}/{}_remaining_points.csv'.format(city,city))
    location_df=pd.read_csv(location)
    location_df['geometry']=location_df.apply(lambda x: Point((float(x.panoLon), float(x.panoLat))), axis=1)
    location_gdf=gpd.GeoDataFrame(location_df, geometry='geometry')
    location_gdf=location_gdf.set_crs(epsg=4326)
    #convert to Pseudo-Mercator
    location_gdf=location_gdf.to_crs("EPSG:3857")
    # merge location to detection result to convert it to gdf
    location_gdf['panoId']='pano='+location_gdf['panoId']
    
    # load land use data
    land_use_gdf=gpd.read_file(os.path.join(root,'data/land_use/{}/land_use.shp'.format(city)))
    land_use_gdf=land_use_gdf.to_crs("EPSG:3857")
    
    # create 500m buffer around img_location_gdf
    buffer_500m = location_gdf.copy()
    buffer_500m["geometry"]=location_gdf.geometry.buffer(500)
    # coduct spatial join to extract points that intersect with the 500m buffer
    land_use_in_500m = gpd.sjoin(buffer_500m, land_use_gdf, how="left", op='intersects')
    # calculate the area
    land_use_in_500m['area']=land_use_in_500m['geometry'].area
    # group by panoId and land use to get sum
    land_use_in_500m=land_use_in_500m.groupby(['panoId','land_use_m']).\
                              agg({'area':'sum'}).\
                              reset_index()
    land_use_in_500m = land_use_in_500m.pivot(index='panoId', columns='land_use_m', values='area').reset_index()
    # fill NaN with 0
    land_use_in_500m=land_use_in_500m.fillna(0)
    land_use_in_500m['total']=land_use_in_500m['residential']+\
                              land_use_in_500m['commercial']+\
                              land_use_in_500m['industrial']
    # calculate proportion of each land use
    land_use_in_500m['residential_percentage']=land_use_in_500m['residential']/land_use_in_500m['total']
    land_use_in_500m['commercial_percentage']=land_use_in_500m['commercial']/land_use_in_500m['total']
    land_use_in_500m['industrial_percentage']=land_use_in_500m['industrial']/land_use_in_500m['total']
    # calculate land_use_mix
    land_use_in_500m['residential_percentage_log']=land_use_in_500m['residential_percentage']*np.log(land_use_in_500m['residential_percentage'])
    land_use_in_500m['commercial_percentage_log']=land_use_in_500m['commercial_percentage']*np.log(land_use_in_500m['commercial_percentage'])
    land_use_in_500m['industrial_percentage_log']=land_use_in_500m['industrial_percentage']*np.log(land_use_in_500m['industrial_percentage'])
    land_use_in_500m['residential_percentage_log']=land_use_in_500m['residential_percentage_log'].fillna(0)
    land_use_in_500m['commercial_percentage_log']=land_use_in_500m['commercial_percentage_log'].fillna(0)
    land_use_in_500m['industrial_percentage_log']=land_use_in_500m['industrial_percentage_log'].fillna(0)
    land_use_in_500m['land_use_mix']=-1*(land_use_in_500m['residential_percentage_log']+\
                                         land_use_in_500m['commercial_percentage_log']+\
                                         land_use_in_500m['industrial_percentage_log'])/(math.log(3))
    land_use_in_500m['land_use_mix']=np.where(land_use_in_500m['land_use_mix'] == -0.0, 0, land_use_in_500m['land_use_mix'])
    
    # merge with the original location_gdf
    land_use_in_500m=pd.merge(location_gdf[['panoId']],
                              land_use_in_500m,
                              on='panoId',
                              how='left'
                             )
    land_use_in_500m=land_use_in_500m.rename(columns={'panoId':'pano_id'})
    land_use_in_500m=land_use_in_500m[['pano_id','land_use_mix']]
    print(land_use_in_500m)
    # save the dataframe to csv
    land_use_in_500m.to_csv(os.path.join(root,'data/tabular_data/{}/land_use.csv'.format(city)))

## Slope

### download data

#### Tokyo

In [ ]:
!wget --user globaldem --password preciseelevation -O /home/ec2-user/SageMaker/data/topography/Tokyo/dem_tif_n30e120.tar http://hydro.iis.u-tokyo.ac.jp/~yamadai/MERIT_DEM/distribute/v1.0.2/dem_tif_n30e120.tar

In [ ]:
# extract it to Tokyo folder
!tar -xf /home/ec2-user/SageMaker/data/topography/dem_tif_n30e120.tar --directory /home/ec2-user/SageMaker/data/topography/Tokyo/

#### Singapore

In [ ]:
!wget --user globaldem --password preciseelevation -O /home/ec2-user/SageMaker/data/topography/Singapore/dem_tif_n00e090.tar http://hydro.iis.u-tokyo.ac.jp/~yamadai/MERIT_DEM/distribute/v1.0.2/dem_tif_n00e090.tar

In [ ]:
# extract it to Singapore folder
!tar -xf /home/ec2-user/SageMaker/data/topography/Singapore/dem_tif_n00e090.tar --directory /home/ec2-user/SageMaker/data/topography/Singapore/

### Check the data and move the appropriate file one level up

In [ ]:
# Tokyo
root = "/home/ec2-user/SageMaker/"
dem=os.path.join(root,'data/topography/Tokyo/dem_tif_n30e120/n35e135_dem.tif')
img = rasterio.open(dem)
show(img)

In [ ]:
# move the file
root = "/home/ec2-user/SageMaker/"
topo_folder=os.path.join(root,'data/topography/')
dem=os.path.join(topo_folder,'Tokyo/dem_tif_n30e120/n35e135_dem.tif')
shutil.copy2(dem,os.path.join(topo_folder,'Tokyo'))

In [ ]:
# Singapore
root = "/home/ec2-user/SageMaker/"
dem=os.path.join(root,'data/topography/Singapore/dem_tif_n00e090/n00e100_dem.tif')
img = rasterio.open(dem)
show(img)

In [ ]:
# move the file
root = "/home/ec2-user/SageMaker/"
topo_folder=os.path.join(root,'data/topography/')
dem=os.path.join(topo_folder,'Singapore/dem_tif_n00e090/n00e100_dem.tif')
shutil.copy2(dem,os.path.join(topo_folder,'Singapore'))

### calculate slope

In [ ]:
city_list=[
    'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# go through cities in the list
for city in city_list:
    dem_list=glob.glob(os.path.join(root,'data/topography/{}/*.tif'.format(city)))
    for dem in dem_list:
        dem_array = rd.LoadGDAL(dem)
        slope = rd.TerrainAttribute(dem, attrib='slope_riserun')
        rd.rdShow(slope, axes=False, cmap='magma', figsize=(8, 5.5))
        plt.show()

### extract slope value to each sample point

In [ ]:
# The code above didn't work, so I imported data from local computer
# use that data to extract slope value to each sample point

city_list=[
    'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# define a function to return a value, given x and y
def getRasterValue(raster,x,y):
    row, col = raster.index(x,y)
    value=raster.read(1)[row,col]
    return value

# go through cities in the list
for city in city_list:
    #open slope file
    slope_file=os.path.join(root,'data/topography/{}/slope_{}.tif'.format(city,city))
    slope = rasterio.open(slope_file)
    
    # sample point locations
    location=os.path.join(root,'data/meta_data_{}/{}_remaining_points.csv'.format(city,city))
    location_df=pd.read_csv(location)
    location_df['geometry']=location_df.apply(lambda x: Point((float(x.panoLon), float(x.panoLat))), axis=1)
    location_gdf=gpd.GeoDataFrame(location_df, geometry='geometry')
    location_gdf=location_gdf.set_crs(epsg=4326)
    # merge location to detection result to convert it to gdf
    location_gdf['panoId']='pano='+location_gdf['panoId']
    # apply getRasterValue function to get slope value from the raster for each sample point
    location_gdf['slope']= location_gdf.apply(lambda x: getRasterValue(slope, x['geometry'].xy[0][0], x['geometry'].xy[1][0]), axis=1)
    # only keep panoId and slope
    location_gdf=location_gdf[['panoId','slope']]
    location_gdf=location_gdf.rename(columns={'panoId':'pano_id'})
    
    # save the data
    print(location_gdf)
    location_gdf.to_csv(os.path.join(root,'data/tabular_data/{}/slope.csv'.format(city)))

## Air quality
- Input cities' names
- use api (https://api.waqi.info/map/bounds/?latlng=&token=) to get station names and their locations (lat&lon)
- use station names to download the file
- conduct everything above locally
- import data on AWS

### clean the data

In [ ]:
# clean the data
city_list=[
    'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# go through cities in the list
for city in city_list:
    # clean the csv file names and station names
    # station names
    station_df=pd.read_csv(os.path.join(root,'data/aqi/{}/{}_aqi_station.csv'.format(city,city)))
    # lower-case all the letters
    station_df['name']=station_df['name'].str.lower()
    # convert ū,ō to u,o
    station_df['name']=station_df['name'].str.replace('ū','u').str.replace('ō','o')
    # convert tokyo-to to tokyo
    station_df['name']=station_df['name'].str.replace('tokyo-to','tokyo')
    # convert ', ' to '_'
    station_df['name']=station_df['name'].str.replace(', ','_')
    # remove '_japan'
    station_df['name']=station_df['name'].str.replace('_japan','')
    # convert ' ' to '_'
    station_df['name']=station_df['name'].str.replace(' ','_')
    # remove '-ku'
    station_df['name']=station_df['name'].str.replace('-ku','')
    # convert '-' to '_'
    station_df['name']=station_df['name'].str.replace('-','_')
    
    # csv file names
    csv_file_list_raw=glob.glob(os.path.join(root,'data/aqi/{}/historical_data/*.csv'.format(city)))
    csv_file_list=[os.path.split(file)[1] for file in csv_file_list_raw]
    # convert ū,ō to u,o
    csv_file_list=[file.replace('ū','u') for file in csv_file_list]
    csv_file_list=[file.replace('ō','o') for file in csv_file_list]
    # convert tokyo-to to tokyo
    csv_file_list=[file.replace('tokyo-to','tokyo') for file in csv_file_list]
    # remove '-air-quality'
    csv_file_list=[file.replace('-air-quality','') for file in csv_file_list]
    # convert '-' to ' '
    csv_file_list=[file.replace(',-','_') for file in csv_file_list]
    # convert ', ' to '_'
    csv_file_list=[file.replace(', ','_') for file in csv_file_list]
    # remove '_japan'
    csv_file_list=[file.replace('_japan','') for file in csv_file_list]  
    # remove '.csv'
    csv_file_list=[file.replace('.csv','') for file in csv_file_list] 
    # remove '-ku'
    csv_file_list=[file.replace('-ku','') for file in csv_file_list] 
    # convert '-' to '_'
    csv_file_list=[file.replace('-', '_') for file in csv_file_list] 
    
    # create a new column to check if the file is extracted
    station_df['extracted']=False
    cond_list=[station_df['name'].isin(csv_file_list)]
    choice_list=[True]
    station_df['extracted']=np.select(cond_list,choice_list)
    print(csv_file_list)
    print(station_df.loc[station_df['extracted']==False])
    
    # save the modified dataframe
    station_df.to_csv(os.path.join(root,'data/aqi/{}/{}_aqi_station_cleaned.csv'.format(city,city)))
    # save the csv file as the modified names
    for i in range(len(csv_file_list_raw)):
        filename=csv_file_list[i]+'.csv'
        os.rename(csv_file_list_raw[i],
                  os.path.join(os.path.split(csv_file_list_raw[i])[0],filename))

### calculate AQI for each station

In [ ]:
# calculate AQI for each station
city_list=[
    'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# create a function to get mean of AQI, given a station name
def getMeanAQI(city, station_name):
    hist_data_df=pd.read_csv(os.path.join(root,'data/aqi/{}/historical_data/{}.csv'.format(city,station_name)))
    # calculate aqi for each day
    hist_data_df["AQI"] =hist_data_df.iloc[:,1:].max(axis=1)
    # only select 2020 data
    hist_data_df= hist_data_df.loc[hist_data_df['date'].str.contains('2020')]
    # calculate the average of the whole data
    aqi_mean=hist_data_df["AQI"].mean()
    return aqi_mean

# go through cities in the list
for city in city_list:
    # load data of station names
    station_df=pd.read_csv(os.path.join(root,'data/aqi/{}/{}_aqi_station_cleaned.csv'.format(city,city)))
    # drop those that didnt extract the aqi
    station_df=station_df.loc[station_df['extracted']==1]
    # drop extreme longitude
    station_df=station_df.loc[station_df['lon']>0]
    # create a aqi_mean column
    station_df['aqi_mean']=station_df['name'].apply(lambda x: getMeanAQI(city, x))
    # drop tokorozawa because it's creating duplicates
    station_df=station_df.loc[station_df['name']!='tokorozawa']
    # save the dataframe
    station_df=station_df[['name','lat','lon','aqi_mean']]
    print(station_df)
    station_df.to_csv(os.path.join(root,'data/aqi/{}/{}_aqi_station_mean_extracted.csv'.format(city,city)))

### conduct spatial interpolation (IDW)
- This couldn't be run, so I ran locally

In [ ]:
# # conduct spatial interpolation (IDW)
# city_list=[
#     'Singapore',
#     'Tokyo'
#           ]
# root = "/home/ec2-user/SageMaker/"

# # go through cities in the list
# for city in city_list:
#     # load mean aqi data at stations
#     station_df=pd.read_csv(os.path.join(root,'data/aqi/{}/{}_aqi_station_mean_extracted.csv'.format(city,city)))
    
#     # sample point locations
#     location=os.path.join(root,'data/meta_data_{}/{}_remaining_points.csv'.format(city,city))
#     location_df=pd.read_csv(location)
#     # merge location to detection result to convert it to gdf
#     location_df['panoId']='pano='+location_df['panoId']
    
#     # conduct IDW
#     lons=np.array(station_df['lon']) 
#     lats=np.array(station_df['lat']) 
#     lon_lat=np.array(list(zip(lons,lats)))
#     zdata=np.array(station_df['aqi_mean'])
#     #test data
#     lons_test=np.array(location_df['panoLon']) 
#     lats_test=np.array(location_df['panoLat']) 
#     lon_lat_test=np.array(list(zip(lons_test,lats_test)))
#     #idw
#     grid_z1 = griddata(lon_lat, zdata, (lons_test, lats_test), method='nearest')
#     location_df['aqi']=grid_z1

#     # save the data
#     location_df=location_df[['panoId','panoLon','panoLat','aqi']]
#     location_df=location_df.rename(columns={'panoId':'pano_id'})
#     print(location_df)
#     location_df.to_csv(os.path.join(root,'data/tabular_data/{}/aqi.csv'.format(city)))

# Extract features for perception prediction
Steps:
    1. Extract features for all the images
    2. Get the features for img_survey based on the list of the pano_id

## Segmentation
*segmentation is a bit different from others because it was conducted in the previous section.

*code below only filters data for img_survey

### 1st survey

In [ ]:
city_list=[
    'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# get segmentation result and combine them
combined_segmentation=pd.DataFrame()
# create output folder
output_folder=os.path.join(root,'data/tabular_data/survey/')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# go through cities in the list
for city in city_list:
    # segmentation
    segmentation_result_file=os.path.join(root,'data/tabular_data/{}/segmentation.csv'.format(city)) 
    segmentation_df=pd.read_csv(segmentation_result_file)
    # change the first col name to pano_id
    segmentation_df.rename(columns={segmentation_df.columns[0]:'pano_id'}, inplace=True)
    
    # get a dataframe of survey image list
    img_survey_list=glob.glob(os.path.join(root,'data/img_survey/{}/*.jpg'.format(city)))
    img_survey_pano_id=[os.path.split(img_survey)[1].replace('.jpg','') for img_survey in img_survey_list]
    img_survey_pano_id_df=pd.DataFrame(img_survey_pano_id,columns=['pano_id'])
    
    # left join the segmentation result to the list of the img_survey pano_id
    img_survey_segmentation=pd.merge(img_survey_pano_id_df,segmentation_df,
                                     on='pano_id',
                                     how='left'
                                    )
    # append the merged df to combined_segmentation
    combined_segmentation=combined_segmentation.append(img_survey_segmentation)

# save it in the output folder as segmentation.csv
combined_segmentation.to_csv(os.path.join(output_folder,'segmentation.csv'))

In [ ]:
# this is only for copying the segmented images to output folder
# # move the segmented img to output
# city_list=[
#     'Singapore',
#     'Tokyo'
#           ]
# root = "/home/ec2-user/SageMaker/"

# # go through cities in the list
# for city in city_list:
#     img_segmented_list=glob.glob(os.path.join(root,'data/img_segmented/{}/*'.format(city)))
#     # copy the segmented_img to output folder
#     output_folder_segmented=os.path.join(root,'output/segmentation/{}/'.format(city))
#     if not os.path.exists(output_folder_segmented):
#         os.makedirs(output_folder_segmented)
#     for img_segmented in img_segmented_list:
#         shutil.copy2(img_segmented, output_folder_segmented)

## detection

### 1st survey

In [ ]:
city_list=[
    'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# get detection result and combine them
combined_detection=pd.DataFrame()
# create output folder
output_folder=os.path.join(root,'data/tabular_data/survey/')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# go through cities in the list
for city in city_list:
    # conduct detection for all the images
    input_img_folder=os.path.join(root,'data/img_stitched_{}/'.format(city))
    output_csv_folder=os.path.join(root,'data/tabular_data/{}/'.format(city))
    output_img_folder=os.path.join(root,'output/detection/{}/'.format(city))
    detection.detect_images(input_img_folder,output_csv_folder,output_img_folder,model='yolo3_darknet53_coco')
    
    # get data for img_survey
    detection_result_file=os.path.join(root,'data/tabular_data/{}/detection.csv'.format(city)) 
    detection_df=pd.read_csv(detection_result_file)
    # change the first col name to pano_id
    detection_df.rename(columns={detection_df.columns[0]:'pano_id'}, inplace=True)
    
    # get a dataframe of survey image list
    img_survey_list=glob.glob(os.path.join(root,'data/img_survey/{}/*.jpg'.format(city)))
    img_survey_pano_id=[os.path.split(img_survey)[1].replace('.jpg','') for img_survey in img_survey_list]
    img_survey_pano_id_df=pd.DataFrame(img_survey_pano_id,columns=['pano_id'])
    
    # left join the detection result to the list of the img_survey pano_id
    img_survey_detection=pd.merge(img_survey_pano_id_df,detection_df,
                                     on='pano_id',
                                     how='left'
                                    )
    # append the merged df to combined_detection
    combined_detection=combined_detection.append(img_survey_detection)
# save it in the output folder as detection.csv
combined_detection.to_csv(os.path.join(output_folder,'detection.csv'))

## classification

### 1st survey

In [ ]:
city_list=['Singapore',
           'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"
model_folder=os.path.join(root,'models/')
# get classification result and combine them
combined_classification=pd.DataFrame()
# create output folder
output_folder=os.path.join(root,'data/tabular_data/survey/')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for city in city_list:
    # conduct classification for all the images
    input_img_folder=os.path.join(root,'data/img_stitched_{}/'.format(city))
    output_folder=os.path.join(root,'data/tabular_data/{}/'.format(city))
    classification.classify_image(model_folder,input_img_folder,output_folder)
    
    # get data for img_survey
    classification_result_file=os.path.join(root,'data/tabular_data/{}/classification.csv'.format(city)) 
    classification_df=pd.read_csv(classification_result_file)
    # change the first col name to pano_id
    classification_df.rename(columns={classification_df.columns[0]:'pano_id'}, inplace=True)
    
    # get a dataframe of survey image list
    img_survey_list=glob.glob(os.path.join(root,'data/img_survey/{}/*.jpg'.format(city)))
    img_survey_pano_id=[os.path.split(img_survey)[1].replace('.jpg','') for img_survey in img_survey_list]
    img_survey_pano_id_df=pd.DataFrame(img_survey_pano_id,columns=['pano_id'])
    
    # left join the classification result to the list of the img_survey pano_id
    img_survey_classification=pd.merge(img_survey_pano_id_df,classification_df,
                                     on='pano_id',
                                     how='left'
                                    )
    # append the merged df to combined_classification
    combined_classification=combined_classification.append(img_survey_classification)
# save it in the output folder as classification.csv
combined_classification.to_csv(os.path.join(output_folder,'classification.csv'))

## edge detection

### 1st survey

In [ ]:
city_list=['Singapore',
           'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"
# get edge_detection result and combine them
combined_edge_detection=pd.DataFrame()
# create output folder
output_folder=os.path.join(root,'data/tabular_data/survey/')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for city in city_list:
    # conduct edge_detection for all the images
    input_img_folder=os.path.join(root,'data/img_stitched_{}/'.format(city))
    output_img_folder=os.path.join(root,'output/edge_detection/{}/'.format(city))
    output_csv_folder=os.path.join(root,'data/tabular_data/{}/'.format(city))
    edge_detection.edge_detect_image(input_img_folder,output_img_folder,output_csv_folder)
    
    # get data for img_survey
    edge_detection_result_file=os.path.join(root,'data/tabular_data/{}/edge_detection.csv'.format(city)) 
    edge_detection_df=pd.read_csv(edge_detection_result_file)
    # change the first col name to pano_id
    edge_detection_df.rename(columns={edge_detection_df.columns[0]:'pano_id'}, inplace=True)
    
    # get a dataframe of survey image list
    img_survey_list=glob.glob(os.path.join(root,'data/img_survey/{}/*.jpg'.format(city)))
    img_survey_pano_id=[os.path.split(img_survey)[1].replace('.jpg','') for img_survey in img_survey_list]
    img_survey_pano_id_df=pd.DataFrame(img_survey_pano_id,columns=['pano_id'])
    
    # left join the edge_detection result to the list of the img_survey pano_id
    img_survey_edge_detection=pd.merge(img_survey_pano_id_df,edge_detection_df,
                                     on='pano_id',
                                     how='left'
                                    )
    # append the merged df to combined_edge_detection
    combined_edge_detection=combined_edge_detection.append(img_survey_edge_detection)
# save it in the output folder as edge_detection.csv
combined_edge_detection.to_csv(os.path.join(output_folder,'edge_detection.csv'))

## blob detection

### 1st survey

In [ ]:
city_list=['Singapore',
           'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"
# get blob_detection result and combine them
combined_blob_detection=pd.DataFrame()
# create output folder
output_folder=os.path.join(root,'data/tabular_data/survey/')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for city in city_list:
    # conduct blob_detection for all the images
    input_img_folder=os.path.join(root,'data/img_stitched_{}/'.format(city))
    output_img_folder=os.path.join(root,'output/blob_detection/{}/'.format(city))
    output_csv_folder=os.path.join(root,'data/tabular_data/{}/'.format(city))
    blob_detection.blob_detect_image(input_img_folder,output_img_folder,output_csv_folder)
    
    # get data for img_survey
    blob_detection_result_file=os.path.join(root,'data/tabular_data/{}/blob_detection.csv'.format(city)) 
    blob_detection_df=pd.read_csv(blob_detection_result_file)
    # change the first col name to pano_id
    blob_detection_df.rename(columns={blob_detection_df.columns[0]:'pano_id'}, inplace=True)
    
    # get a dataframe of survey image list
    img_survey_list=glob.glob(os.path.join(root,'data/img_survey/{}/*.jpg'.format(city)))
    img_survey_pano_id=[os.path.split(img_survey)[1].replace('.jpg','') for img_survey in img_survey_list]
    img_survey_pano_id_df=pd.DataFrame(img_survey_pano_id,columns=['pano_id'])
    
    # left join the blob_detection result to the list of the img_survey pano_id
    img_survey_blob_detection=pd.merge(img_survey_pano_id_df,blob_detection_df,
                                     on='pano_id',
                                     how='left'
                                    )
    # append the merged df to combined_blob_detection
    combined_blob_detection=combined_blob_detection.append(img_survey_blob_detection)
# save it in the output folder as blob_detection.csv
combined_blob_detection.to_csv(os.path.join(output_folder,'blob_detection.csv'))

## HLS

### 1st survey

In [ ]:
city_list=['Singapore',
           'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"
# get hls_statistics result and combine them
combined_hls_statistics=pd.DataFrame()
# create output folder
output_folder=os.path.join(root,'data/tabular_data/survey/')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for city in city_list:
    # conduct hls_statistics for all the images
    input_img_folder=os.path.join(root,'data/img_stitched_{}/'.format(city))
    output_csv_folder=os.path.join(root,'data/tabular_data/{}/'.format(city))
    hls_statistics.blob_detect_image(input_img_folder,output_csv_folder)
    
    # get data for img_survey
    hls_statistics_result_file=os.path.join(root,'data/tabular_data/{}/hls_statistics.csv'.format(city)) 
    hls_statistics_df=pd.read_csv(hls_statistics_result_file)
    # change the first col name to pano_id
    hls_statistics_df.rename(columns={hls_statistics_df.columns[0]:'pano_id'}, inplace=True)
    
    # get a dataframe of survey image list
    img_survey_list=glob.glob(os.path.join(root,'data/img_survey/{}/*.jpg'.format(city)))
    img_survey_pano_id=[os.path.split(img_survey)[1].replace('.jpg','') for img_survey in img_survey_list]
    img_survey_pano_id_df=pd.DataFrame(img_survey_pano_id,columns=['pano_id'])
    
    # left join the hls_statistics result to the list of the img_survey pano_id
    img_survey_hls_statistics=pd.merge(img_survey_pano_id_df,hls_statistics_df,
                                     on='pano_id',
                                     how='left'
                                    )
    # append the merged df to combined_hls_statistics
    combined_hls_statistics=combined_hls_statistics.append(img_survey_hls_statistics)
# save it in the output folder as hls_statistics.csv
combined_hls_statistics.to_csv(os.path.join(output_folder,'hls_statistics.csv'))

## Data organization

#### Outlier detection 

In [ ]:
# load data
root = "/home/ec2-user/SageMaker/"
result=pd.read_csv(os.path.join(root,'data/survey_result/Batch_4364547_batch_results.csv'))

# clean the data
result_clean=result[['WorkerId','WorkTimeInSeconds','Input.url','Answer.beauty',
               'Answer.building_attractiveness','Answer.cleanliness','Answer.cycling_attractiveness',
               'Answer.living_attractiveness','Answer.safety','Answer.spaciousness']]
result_clean.columns=[col.replace('Answer.','',) for col in result_clean.columns]
result_clean['panoId']=result_clean['Input.url'].\
    str.replace('https://gsv-perception-survey-img.s3-ap-southeast-1.amazonaws.com/data/','').\
    str.replace('pano%3D','pano=').\
    str[:-4]
result_clean=result_clean.drop(['Input.url'],axis=1)

# group by workerid to get mean worktime and range of scale used
result_clean_grouped=result_clean.groupby(['WorkerId']).\
    agg({'WorkTimeInSeconds':['mean'],
         'beauty':['min','max'],
         'building_attractiveness':['min', 'max'],
         'cleanliness':['min', 'max'],
         'cycling_attractiveness':['min', 'max'],
         'living_attractiveness':['min', 'max'],
         'safety':['min', 'max'],
         'spaciousness':['min', 'max']}).\
    reset_index()
# get the col names of min&max values
result_clean_grouped.columns=result_clean_grouped.columns.map(lambda t: t[0] + "_" + t[1])
perception_score_min_cols=result_clean_grouped.columns[range(2, 16, 2)]
result_clean_grouped['overall_min']=result_clean_grouped[perception_score_min_cols].min(axis=1)
perception_score_max_cols=result_clean_grouped.columns[range(3, 17, 2)]
result_clean_grouped['overall_max']=result_clean_grouped[perception_score_max_cols].max(axis=1)
result_clean_grouped['scale_range']=result_clean_grouped['overall_max']-result_clean_grouped['overall_min']

# visualize them
ax=sns.boxplot(data=result_clean_grouped, y="WorkTimeInSeconds_mean")
ax.set_title("Box Plot of Work Time by Participants", fontdict={"fontsize": "25", "fontweight" : "3"})
plt.savefig(os.path.join(root, 'output/survey_result/work_time_boxplot.jpg'),bbox_inches = "tight",dpi=400)
ax=sns.boxplot(data=result_clean_grouped, y="scale_range")
ax.set_title("Box Plot of Range of Scale by Participants", fontdict={"fontsize": "25", "fontweight" : "3"})
plt.savefig(os.path.join(root, 'output/survey_result/scale_range_boxplot.jpg'),bbox_inches = "tight",dpi=400)

In [ ]:
def mahalanobis_method(df):
    #M-Distance
    x_minus_mu = df - np.mean(df)
    cov = np.cov(df.values.T)                           #Covariance
    inv_covmat = sp.linalg.inv(cov)                     #Inverse covariance
    left_term = np.dot(x_minus_mu, inv_covmat) 
    mahal = np.dot(left_term, x_minus_mu.T)
    md = np.sqrt(mahal.diagonal())
    
    #Flag as outlier
    outlier = []
    #Cut-off point
    C = np.sqrt(chi2.ppf((1-0.001), df=df.shape[1]))    #degrees of freedom = number of variables
    for index, value in enumerate(md):
        if value > C:
            outlier.append(index)
        else:
            continue
    return outlier, md

In [ ]:
# use MD
worktime_scale_range=result_clean_grouped[["WorkTimeInSeconds_mean","scale_range"]]
outliers_mahal_bi, md_bi = mahalanobis_method(df=worktime_scale_range)

#Visualization
#add column to a slice of a DataFrame
df_bi_cp = copy.deepcopy(worktime_scale_range) 

#Add md and robust md to copy of dataframe
df_bi_cp['md'] = md_bi

def flag_outliers(df, outliers):
    flag = []
    for index in range(df.shape[0]):
        if index in outliers:
            flag.append(1)
        else:
            flag.append(0)
    return flag

#Flag outliers with 1, others with 0
df_bi_cp['flag'] = flag_outliers(worktime_scale_range, outliers_mahal_bi)

#show and save
root = "/home/ec2-user/SageMaker/"
ax = sns.scatterplot(x="WorkTimeInSeconds_mean", y="scale_range", hue='flag', data=df_bi_cp)
plt.savefig(os.path.join(root, 'output/survey_result/worktime_scale_outlier.jpg'),bbox_inches = "tight",dpi=400)

### combine data

In [ ]:
# take average of the survey result by panoid
root = "/home/ec2-user/SageMaker/"
result=pd.read_csv(os.path.join(root,'data/survey_result/Batch_4364547_batch_results.csv'))
result_clean=result.iloc[:,27:35]
result_clean.columns=[col.replace('Answer.','',) for col in result_clean.columns]
result_clean['panoId']=result_clean['Input.url'].\
    str.replace('https://gsv-perception-survey-img.s3-ap-southeast-1.amazonaws.com/data/','').\
    str.replace('pano%3D','pano=').\
    str[:-4]
result_clean=result_clean[['panoId','beauty','building_attractiveness','cleanliness','cycling_attractiveness',
                          'living_attractiveness','safety','spaciousness']]
# take average by panoId
result_clean=result_clean.groupby(['panoId']).\
    agg({'beauty':'mean',
         'building_attractiveness':'mean',
         'cleanliness':'mean',
         'cycling_attractiveness':'mean',
         'living_attractiveness':'mean',
         'safety':'mean',
         'spaciousness':'mean'}).\
    reset_index()
# save the grouped result
result_clean=result_clean.rename(columns={'panoId':'pano_id'})
result_clean.to_csv(os.path.join(root,'data/survey_result/grouped_survey_result.csv'))

In [ ]:
city_list=['Singapore',
           'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

data_list=['segmentation',
           'detection',
           'classification',
           'edge_detection',
           'blob_detection',
           'hls_statistics'
          ]

# survey result
result=pd.read_csv(os.path.join(root,'data/survey_result/grouped_survey_result.csv'))

# go through all the dataset
for data in data_list:
    df_temp=pd.DataFrame()
    for city in city_list:
        # load the data
        data_df=pd.read_csv(os.path.join(root,'data/tabular_data/{}/{}.csv'.format(city,data)))
        # append the data_df to df_temp
        df_temp=df_temp.append(data_df)
    # left join the df_temp to result
    if not 'pano_id' in df_temp.columns:
        df_temp=df_temp.rename(columns={df_temp.columns[0]:'pano_id'})
    df_temp.drop(df_temp.filter(regex='Unnamed').columns, axis=1, inplace=True)
    df_temp.columns=['pano_id' if col =='pano_id' else data+'_'+col for col in df_temp.columns]
    result=pd.merge(result,df_temp,on='pano_id',how='left')
result.drop(result.filter(regex='Unnamed').columns, axis=1, inplace=True)
# save the combined data
result.to_csv(os.path.join(root,'data/survey_result/survey_result_with_independent_variables.csv'))

## Predict perception

#### Hyper parameter tuning

In [ ]:
root = "/home/ec2-user/SageMaker/"

# Global_vars
seed = 1234
num_folds = 10
n_jobs = 3
scoring='neg_root_mean_squared_error'
model_output_folder=os.path.join(root,'models/')

# load the data
result=pd.read_csv(os.path.join(root,'data/survey_result/survey_result_with_independent_variables.csv'))

# create a list of target variables
target_variables=['beauty',
                  'building_attractiveness','cleanliness','cycling_attractiveness',
                  'living_attractiveness','safety','spaciousness'
                 ]
for target_variable in tqdm.tqdm(target_variables):
    X=result.iloc[:,9:]
    y = result[target_variable]
    # split X,y into train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)

    # Create pipelines
    pipelines = pipeline_utils.create_pipelines(seed)

#     # Run cv experiment without hyper_param_tuning
#     results_df = run_cv_and_test(X_train, y_train, X_test, y_test, pipelines, scoring, seed, num_folds,
#                                  dataset_name=parser.name, n_jobs=n_jobs)

#     # Save cv experiment to csv
#     if is_save_results:
#         dataset_results_name = parser.name + "_results.csv"
#         results_path = os.path.join("..", "..", "data", "processed", dataset_results_name)
#         results_df.to_csv(results_path, index=False)


    # Run same experiment with hypertuned parameters
#     print("#"*30 + "Hyper tuning parameters" "#"*30)
    hypertuned_params = pipeline_utils.get_hypertune_params()
    hypertune_results_df = pipeline_utils.run_cv_and_test_hypertuned_params(X_train, y_train, X_test, y_test, 
                                                                            pipelines, scoring, seed, num_folds,
                                                                            hypertuned_params, target_variable, n_jobs, 
                                                                            model_output_folder)
    print(hypertune_results_df)

### predict for the rest of the data (including the ones surveyed)

#### create models using the best parameters

In [ ]:
# # create a list of target variables
# target_variables=['beauty',
#                   'building_attractiveness','cleanliness','cycling_attractiveness',
#                   'living_attractiveness','safety','spaciousness'
#                  ]
# model_name='LGBMR-PCA'

# model=lgb.train(params,
#                 dtrain,
#                 valid_sets=[dtrain,dval],
#                 verbose_eval=100,
#                 early_stopping_rounds=100
#                 )

# prediction=np.rint(model.predict(X_test,num_iteration=model.best_iteration))

In [ ]:
city_list=['Singapore',
           'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

model_output_folder=os.path.join(root,'models/')
model_name='StandardScaler_LGBMR-PCA'

data_list=['segmentation',
           'detection',
           'classification',
           'edge_detection',
           'blob_detection',
           'hls_statistics'
          ]

# create a list of target variables
target_variables=['beauty',
                  'building_attractiveness','cleanliness','cycling_attractiveness',
                  'living_attractiveness','safety','spaciousness'
                 ]

for city in city_list:
    # load a dataframe of panoid
    sample_points=pd.read_csv(os.path.join(root,'data/meta_data_{}/{}_remaining_points.csv'.format(city,city)))
    sample_points=sample_points[['panoId']]
    sample_points=sample_points.rename(columns={'panoId':'pano_id'})
    sample_points['pano_id']='pano='+sample_points['pano_id']
    sample_points_keep=sample_points.copy()
    
    # go through data_list and left join the data to sample_points df
    for data in data_list:
        # load the data
        data_df=pd.read_csv(os.path.join(root,'data/tabular_data/{}/{}.csv'.format(city,data)))
        # left join the df_temp to result
        if not 'pano_id' in data_df.columns:
            data_df=data_df.rename(columns={data_df.columns[0]:'pano_id'})
        data_df.drop(data_df.filter(regex='Unnamed').columns, axis=1, inplace=True)
        sample_points=pd.merge(sample_points,data_df,
                              on='pano_id',how='left')
    
    # get X
    X=sample_points.iloc[:,1:]
    print(X)
    
    # go through the target variables
    for target_variable in tqdm.tqdm(target_variables):
        # load the model
        model= joblib.load(os.path.join(model_output_folder,target_variable+"__"+model_name+'.joblib'))
        prediction=model.predict(X)
        prediction_df = pd.DataFrame(data=prediction, columns=["predicted_"+target_variable])
        sample_points_keep_copy=sample_points_keep.copy()
        concatenated_dataframes = pd.concat([sample_points_keep_copy, prediction], axis=1)
        print(concatenated_dataframes)
        
        # save the dataframe
        concatenated_dataframes.to_csv(os.path.join(root,'data/tabular_data/{}/predicted_{}.csv'.format(city,target_variable)))

# 2nd survey
## Select 800 images and upload it to S3 to redo the survey

In [ ]:
# take average of the survey result by panoid
root = "/home/ec2-user/SageMaker/"
result=pd.read_csv(os.path.join(root,'data/survey_result/Batch_4364547_batch_results.csv'))
result_clean=result.iloc[:,27:35]
result_clean.columns=[col.replace('Answer.','',) for col in result_clean.columns]
result_clean['panoId']=result_clean['Input.url'].\
    str.replace('https://gsv-perception-survey-img.s3-ap-southeast-1.amazonaws.com/data/','').\
    str.replace('pano%3D','pano=').\
    str[:-4]
result_clean=result_clean[['panoId','beauty','building_attractiveness','cleanliness','cycling_attractiveness',
                          'living_attractiveness','safety','spaciousness']]
# take average by min & max
result_clean=result_clean.groupby(['panoId']).\
    agg({'beauty':['median', 'min', 'max'],
         'building_attractiveness':['median', 'min', 'max'],
         'cleanliness':['median', 'min', 'max'],
         'cycling_attractiveness':['median', 'min', 'max'],
         'living_attractiveness':['median', 'min', 'max'],
         'safety':['median', 'min', 'max'],
         'spaciousness':['median', 'min', 'max']}).\
    reset_index()

# calculate the difference between min max
result_clean=result_clean.rename(columns={'panoId':'pano_id'})
# create a list of target variables
target_variables=['beauty',
                  'building_attractiveness','cleanliness','cycling_attractiveness',
                  'living_attractiveness','safety','spaciousness']
for target_variable in target_variables:
    result_clean[(target_variable,'min_max_dif')]=result_clean[(target_variable,'max')]-result_clean[(target_variable,'min')]
    
# merge with city names
result_clean.columns=result_clean.columns.map(lambda t: t[0] + "_" + t[1])
result_clean.columns=result_clean.columns.map(lambda t: t.strip('_'))
city_list=['Singapore',
           'Tokyo'
          ]
for city in city_list:
    # location data
    location=os.path.join(root,'data/meta_data_{}/{}_remaining_points.csv'.format(city,city))
    location_df=pd.read_csv(location)
    location_df['city']=city
    location_df=location_df.rename(columns={'panoId':'pano_id'})
    location_df['pano_id']='pano='+location_df['pano_id']
    # merge with the result_clean
    result_clean=pd.merge(result_clean,location_df[['pano_id','city']],
                         on='pano_id',how='left')
# merge city_x and city_y and rename it to city
result_clean['city_x']=result_clean['city_x'].fillna(result_clean['city_y'])
result_clean=result_clean.rename(columns={'city_x':'city'})

# filter the data to choose smaller min_max_dif
result_clean_min_max_dif=result_clean.iloc[:,[0,22,23,24,25,26,27,28,29]]
result_clean_city_filtered=pd.DataFrame()
for city in city_list:
    df_temp=result_clean.loc[result_clean['city']==city]
    df_temp_min_max_dif_less_than_5=df_temp.loc[(df_temp['beauty_min_max_dif']<5)&\
                                              (df_temp['building_attractiveness_min_max_dif']<5)&\
                                              (df_temp['cleanliness_min_max_dif']<5)&\
                                              (df_temp['cycling_attractiveness_min_max_dif']<5)&\
                                              (df_temp['living_attractiveness_min_max_dif']<5)&\
                                              (df_temp['safety_min_max_dif']<5)&\
                                              (df_temp['spaciousness_min_max_dif']<5)
                                               ]
    # randomly sample 400 images
    n=400
    seed=1234
    df_temp_min_max_dif_less_than_5_400=df_temp_min_max_dif_less_than_5.sample(n=n,random_state=seed)
    # append to result_clean_city_filtered
    result_clean_city_filtered=result_clean_city_filtered.append(df_temp_min_max_dif_less_than_5_400)

# upload it to S3
# set up sagemaker session
role = get_execution_role()
print(role)
sess = sagemaker.Session()
print(sess)

# create a s3 bucket
bucket_name='gsv-perception-survey-img'
key_prefix='data_resampled_800'
s3 = boto3.client('s3')
my_region = boto3.session.Session().region_name
try:
    if  my_region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

# go through city_list
for city in city_list:
    # sample 1,500 images
    result_city_specific=result_clean_city_filtered.loc[result_clean_city_filtered['city']==city]
    img_list=[os.path.join(root,'data/img_stitched_{}/{}.jpg'.format(city,row['pano_id']))\
              for index,row in result_city_specific.iterrows()]
    # create a local output folder
    out_folder=os.path.join(root,'data/img_survey_resampled_800/{}/'.format(city))
    if not os.path.exists(out_folder):
        os.makedirs(out_folder)
    
    # save in img_survey folder and upload the data to bucket
    for img in tqdm.tqdm(img_list):
        # save locally
        shutil.copy2(img, out_folder)
        
        # upload to s3
        sess.upload_data(path=img, bucket=bucket_name,key_prefix=key_prefix)

# Create a csv file for s3 URL
url_list=[]
outputFolder=os.path.join(root,'data/tabular_data/')

# go through all the objects in the bucket
paginator = s3.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket=bucket_name, Prefix=key_prefix)

for page in pages:
    for obj in page['Contents']:
        # convert file names into url
        file_modified=obj['Key'].replace('=','%3D')
        # create url
        url='https://{BUCKET_NAME}.s3-ap-southeast-1.amazonaws.com/{FILE_NAME}'.\
        format(BUCKET_NAME=bucket_name,FILE_NAME=file_modified)
        url_list.append(url)
url_df=pd.DataFrame(url_list,columns=['url'])
url_df.to_csv(os.path.join(outputFolder,'survey_resampled_800_aws_url.csv'), index=False)

## segmentation

In [ ]:
city_list=[
    'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# get segmentation result and combine them
combined_segmentation=pd.DataFrame()
# create output folder
output_folder=os.path.join(root,'data/tabular_data/survey_2nd/')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# go through cities in the list
for city in city_list:
    # segmentation
    segmentation_result_file=os.path.join(root,'data/tabular_data/{}/segmentation.csv'.format(city)) 
    segmentation_df=pd.read_csv(segmentation_result_file)
    # change the first col name to pano_id
    segmentation_df.rename(columns={segmentation_df.columns[0]:'pano_id'}, inplace=True)
    
    # get a dataframe of survey image list
    img_survey_list=glob.glob(os.path.join(root,'data/img_survey_resampled_800/{}/*.jpg'.format(city)))
    img_survey_pano_id=[os.path.split(img_survey)[1].replace('.jpg','') for img_survey in img_survey_list]
    img_survey_pano_id_df=pd.DataFrame(img_survey_pano_id,columns=['pano_id'])
    
    # left join the segmentation result to the list of the img_survey pano_id
    img_survey_segmentation=pd.merge(img_survey_pano_id_df,segmentation_df,
                                     on='pano_id',
                                     how='left'
                                    )
    # append the merged df to combined_segmentation
    combined_segmentation=combined_segmentation.append(img_survey_segmentation)

# save it in the output folder as segmentation.csv
combined_segmentation.to_csv(os.path.join(output_folder,'segmentation.csv'))

## detection

In [ ]:
city_list=[
    'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# get detection result and combine them
combined_detection=pd.DataFrame()
# create output folder
output_folder=os.path.join(root,'data/tabular_data/survey_2nd/')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# go through cities in the list
for city in city_list:
    # detection
    detection_result_file=os.path.join(root,'data/tabular_data/{}/detection.csv'.format(city)) 
    detection_df=pd.read_csv(detection_result_file)
    # change the first col name to pano_id
    detection_df.rename(columns={detection_df.columns[0]:'pano_id'}, inplace=True)
    
    # get a dataframe of survey image list
    img_survey_list=glob.glob(os.path.join(root,'data/img_survey_resampled_800/{}/*.jpg'.format(city)))
    img_survey_pano_id=[os.path.split(img_survey)[1].replace('.jpg','') for img_survey in img_survey_list]
    img_survey_pano_id_df=pd.DataFrame(img_survey_pano_id,columns=['pano_id'])
    
    # left join the detection result to the list of the img_survey pano_id
    img_survey_detection=pd.merge(img_survey_pano_id_df,detection_df,
                                     on='pano_id',
                                     how='left'
                                    )
    # append the merged df to combined_detection
    combined_detection=combined_detection.append(img_survey_detection)

# save it in the output folder as detection.csv
combined_detection.to_csv(os.path.join(output_folder,'detection.csv'))

## classification

In [ ]:
city_list=[
    'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# get classification result and combine them
combined_classification=pd.DataFrame()
# create output folder
output_folder=os.path.join(root,'data/tabular_data/survey_2nd/')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# go through cities in the list
for city in city_list:
    # classification
    classification_result_file=os.path.join(root,'data/tabular_data/{}/classification.csv'.format(city)) 
    classification_df=pd.read_csv(classification_result_file)
    # change the first col name to pano_id
    classification_df.rename(columns={classification_df.columns[0]:'pano_id'}, inplace=True)
    
    # get a dataframe of survey image list
    img_survey_list=glob.glob(os.path.join(root,'data/img_survey_resampled_800/{}/*.jpg'.format(city)))
    img_survey_pano_id=[os.path.split(img_survey)[1].replace('.jpg','') for img_survey in img_survey_list]
    img_survey_pano_id_df=pd.DataFrame(img_survey_pano_id,columns=['pano_id'])
    
    # left join the classification result to the list of the img_survey pano_id
    img_survey_classification=pd.merge(img_survey_pano_id_df,classification_df,
                                     on='pano_id',
                                     how='left'
                                    )
    # append the merged df to combined_classification
    combined_classification=combined_classification.append(img_survey_classification)

# save it in the output folder as classification.csv
combined_classification.to_csv(os.path.join(output_folder,'classification.csv'))

## edge detection

In [ ]:
city_list=[
    'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# get edge_detection result and combine them
combined_edge_detection=pd.DataFrame()
# create output folder
output_folder=os.path.join(root,'data/tabular_data/survey_2nd/')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# go through cities in the list
for city in city_list:
    # edge_detection
    edge_detection_result_file=os.path.join(root,'data/tabular_data/{}/edge_detection.csv'.format(city)) 
    edge_detection_df=pd.read_csv(edge_detection_result_file)
    
    # get a dataframe of survey image list
    img_survey_list=glob.glob(os.path.join(root,'data/img_survey_resampled_800/{}/*.jpg'.format(city)))
    img_survey_pano_id=[os.path.split(img_survey)[1].replace('.jpg','') for img_survey in img_survey_list]
    img_survey_pano_id_df=pd.DataFrame(img_survey_pano_id,columns=['pano_id'])
    
    # left join the edge_detection result to the list of the img_survey pano_id
    img_survey_edge_detection=pd.merge(img_survey_pano_id_df,edge_detection_df,
                                     on='pano_id',
                                     how='left'
                                    )
    # append the merged df to combined_edge_detection
    combined_edge_detection=combined_edge_detection.append(img_survey_edge_detection)

# save it in the output folder as edge_detection.csv
combined_edge_detection.to_csv(os.path.join(output_folder,'edge_detection.csv'))

## blob detection

In [ ]:
city_list=[
    'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# get blob_detection result and combine them
combined_blob_detection=pd.DataFrame()
# create output folder
output_folder=os.path.join(root,'data/tabular_data/survey_2nd/')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# go through cities in the list
for city in city_list:
    # blob_detection
    blob_detection_result_file=os.path.join(root,'data/tabular_data/{}/blob_detection.csv'.format(city)) 
    blob_detection_df=pd.read_csv(blob_detection_result_file)
    
    # get a dataframe of survey image list
    img_survey_list=glob.glob(os.path.join(root,'data/img_survey_resampled_800/{}/*.jpg'.format(city)))
    img_survey_pano_id=[os.path.split(img_survey)[1].replace('.jpg','') for img_survey in img_survey_list]
    img_survey_pano_id_df=pd.DataFrame(img_survey_pano_id,columns=['pano_id'])
    
    # left join the blob_detection result to the list of the img_survey pano_id
    img_survey_blob_detection=pd.merge(img_survey_pano_id_df,blob_detection_df,
                                     on='pano_id',
                                     how='left'
                                    )
    # append the merged df to combined_blob_detection
    combined_blob_detection=combined_blob_detection.append(img_survey_blob_detection)

# save it in the output folder as blob_detection.csv
combined_blob_detection.to_csv(os.path.join(output_folder,'blob_detection.csv'))

## HLS

In [ ]:
city_list=[
    'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# get hls_statistics result and combine them
combined_hls_statistics=pd.DataFrame()
# create output folder
output_folder=os.path.join(root,'data/tabular_data/survey_2nd/')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# go through cities in the list
for city in city_list:
    # hls_statistics
    hls_statistics_result_file=os.path.join(root,'data/tabular_data/{}/hls_statistics.csv'.format(city)) 
    hls_statistics_df=pd.read_csv(hls_statistics_result_file)
    
    # get a dataframe of survey image list
    img_survey_list=glob.glob(os.path.join(root,'data/img_survey_resampled_800/{}/*.jpg'.format(city)))
    img_survey_pano_id=[os.path.split(img_survey)[1].replace('.jpg','') for img_survey in img_survey_list]
    img_survey_pano_id_df=pd.DataFrame(img_survey_pano_id,columns=['pano_id'])
    
    # left join the hls_statistics result to the list of the img_survey pano_id
    img_survey_hls_statistics=pd.merge(img_survey_pano_id_df,hls_statistics_df,
                                     on='pano_id',
                                     how='left'
                                    )
    # append the merged df to combined_hls_statistics
    combined_hls_statistics=combined_hls_statistics.append(img_survey_hls_statistics)

# save it in the output folder as hls_statistics.csv
combined_hls_statistics.to_csv(os.path.join(output_folder,'hls_statistics.csv'))

## Get 2nd survey results and combine with the 1st survey results

In [ ]:
city_list=[
    'Singapore',
    'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# load the 1st and 2nd srvey results
result_2nd=pd.read_csv(os.path.join(root,'data/survey_result/Batch_4385106_batch_results.csv'))
result_1st=result=pd.read_csv(os.path.join(root,'data/survey_result/Batch_4364547_batch_results.csv'))
# convert Input.url to pano_id
# 2nd
result_2nd['pano_id']=result_2nd['Input.url'].\
    str.replace('https://gsv-perception-survey-img.s3-ap-southeast-1.amazonaws.com/data_resampled_800/','').\
    str.replace('pano%3D','pano=').\
    str[:-4]
result_2nd=result_2nd.drop(['Input.url'],axis=1)
# 1st
result_1st['pano_id']=result_1st['Input.url'].\
    str.replace('https://gsv-perception-survey-img.s3-ap-southeast-1.amazonaws.com/data/','').\
    str.replace('pano%3D','pano=').\
    str[:-4]
result_1st=result_1st.drop(['Input.url'],axis=1)

# get unique pano_id of result_2nd
result_2nd_unique_pano_list=result_2nd['pano_id'].unique()
# select result_1st with pano_id in the result_2nd_unique_pano
result_1st_for_2nd=result_1st.loc[result_1st['pano_id'].isin(result_2nd_unique_pano_list)]
# append result_1st_for_2nd to result_2nd
result_2nd=result_2nd.append(result_1st_for_2nd)
# save result_2nd
result_2nd.to_csv(os.path.join(root,'data/survey_result/survey_1st_2nd_combined.csv'))

## detect outliers

In [ ]:
def mad_method(df, variable_name):
    #Takes two parameters: dataframe & variable of interest as string
    columns = df.columns
    med = np.median(df, axis = 0)
    mad = np.abs(stats.median_abs_deviation(df))
    threshold = 3
    outlier = []
    index=0
    for item in range(len(columns)):
        if columns[item] == variable_name:
            index == item
    for i, v in enumerate(df.loc[:,variable_name]):
        t = (v-med[index])/mad[index]
        if t > threshold:
            outlier.append(i)
        else:
            continue
    return outlier

In [ ]:
# load data
root = "/home/ec2-user/SageMaker/"
result_1st_2nd=pd.read_csv(os.path.join(root,'data/survey_result/survey_1st_2nd_combined.csv'))

# clean the data
result_clean=result_1st_2nd[['pano_id','Answer.beauty',
               'Answer.building_attractiveness','Answer.cleanliness','Answer.cycling_attractiveness',
               'Answer.living_attractiveness','Answer.safety','Answer.spaciousness']]
result_clean.columns=[col.replace('Answer.','',) for col in result_clean.columns]

# get unique pano_id
result_clean_unique_pano_list=result_clean['pano_id'].unique()
# go through each pano_id to compute outlier
# store all the index in a dictionary
outlier_index_dict={}
for i in range(len(result_clean.columns)-1):
    outlier_index_dict[result_clean.columns[i+1]]=[]
    
for pano in tqdm.tqdm(result_clean_unique_pano_list):
    result_clean_pano=result_clean.loc[result_clean['pano_id']==pano]
    result_clean_pano=result_clean_pano.drop(['pano_id'],axis=1)
    for i in range(len(result_clean.columns)-1):
        outlier_index_local=mad_method(result_clean_pano, result_clean.columns[1+i])
        outlier=result_clean_pano.iloc[outlier_index_local]
        outlier_index_gobal=outlier.index.tolist()
        outlier_index_dict[result_clean.columns[i+1]].extend(outlier_index_gobal)
        
# go through each score in outlier_index_dict to exclude the outliers from the data
# and save the data seperately by score
result_unique_pano_df=pd.DataFrame(result_clean_unique_pano_list,columns=['pano_id'])
for key in outlier_index_dict:
    # exclude outliers
    result_clean_only_key=result_clean.loc[:,['pano_id',key]]
    result_clean_only_key.drop(outlier_index_dict[key], inplace=True)
    
    # calculate mean values
    result_clean_only_key=result_clean_only_key.groupby(['pano_id']).agg({key:'mean'}).reset_index()
    
    # left join the dataframe to result_unique_pano_df
    result_unique_pano_df=pd.merge(result_unique_pano_df,result_clean_only_key,
                                  on='pano_id',how='left')
# save result_unique_pano_df
print(result_unique_pano_df)
result_unique_pano_df.to_csv(os.path.join(root,'data/survey_result/survey_1st_2nd_combined_without_outliers_grouped.csv'))

## Combine data & feature engineering
- Segmentation
    - tree_ss
    - sky_ss
    - street_ss
    - built_ss
    - others_ss
    - nature
    - shannon
- Classification
    - slum_ic 
    - market_ic
    - built_other_ic
    - green_other_ic
- Detection
    - bicycle_od
    - bus_od
    - car_od
    - motorcycle_od
    - person_od
    - traffic_light_od
    - truck_od
- Edge detection
    - canny_edge_llf
- Blob detection
    - no_of_blobs_llf
- HLS
    - hue_mean_llf
    - hue_std_llf
    - lightness_mean_llf
    - lightness_std_llf
    - saturation_mean_llf
    - saturation_std_llf

In [ ]:
root = "/home/ec2-user/SageMaker/"

data_list=['segmentation',
           'detection',
           'classification',
           'edge_detection',
           'blob_detection',
           'hls_statistics'
          ]

# survey result
result=pd.read_csv(os.path.join(root,'data/survey_result/survey_1st_2nd_combined_without_outliers_grouped.csv'))

# go through all the dataset
for data in data_list:
    data_df=pd.read_csv(os.path.join(root,'data/tabular_data/survey_2nd/{}.csv'.format(data)))
    data_df.drop(data_df.filter(regex='Unnamed').columns, axis=1, inplace=True)
    data_df.columns=['pano_id' if col =='pano_id' else data+'_'+col for col in data_df.columns]
    result=pd.merge(result,data_df,on='pano_id',how='left')
result.drop(result.filter(regex='Unnamed').columns, axis=1, inplace=True)

# feature engineering 
def featureEngineer(result,num_id_col):
    # move relevant columns from result to result_final
    result_final=result.iloc[:,0:num_id_col]
    # Segmentation
    seg_col_list=['segmentation_nature--vegetation',
                  'segmentation_nature--sky',
                  'segmentation_construction--flat--road',
                  'segmentation_construction--flat--sidewalk',
                  'segmentation_construction--flat--bike-lane',
                  'segmentation_construction--flat--crosswalk-plain',
                  'segmentation_construction--flat--curb-cut',
                  'segmentation_construction--flat--parking',
                  'segmentation_construction--flat--pedestrian-area',
                  'segmentation_construction--flat--service-lane',
                  'segmentation_construction--structure--building'
                 ]
    nature_col_list=["segmentation_nature", "segmentation_animal"]
    result_final['tree_ss']=result[seg_col_list[0]]
    result_final['sky_ss']=result[seg_col_list[1]]
    result_final['street_ss']=result[seg_col_list[2:10]].sum(axis=1)
    result_final['built_ss']=result[seg_col_list[10]]
    result_seg=result[result.columns[pd.Series(result.columns).str.startswith('segmentation')]]
    result_final['others_ss']=result_seg[result_seg.columns.difference(seg_col_list)].sum(axis=1)
    result_final['nature']=result[result.columns[result.columns.str.startswith(tuple(nature_col_list))]].sum(axis=1)
    result_shannon=result_seg.apply(lambda x: x*np.log(x))
    result_shannon=result_shannon.fillna(0)
    result_final['shannon']=(-1*(result_shannon.sum(axis=1)))/(math.log(len(result.columns)-1))
    # Classification
    result_final['slum_ic']=result[['classification_slum',
                                    'classification_alley',
                                    'classification_junkyard'
                                   ]].sum(axis=1)
    result_final['market_ic']=result[['classification_bazaar/indoor',
                                    'classification_bazaar/outdoor',
                                    'classification_flea_market/indoor',
                                    'classification_market/outdoor'
                                   ]].sum(axis=1)
    result_final['built_other_ic']=result[['classification_downtown',
                                    'classification_embassy',
                                    'classification_plaza'
                                   ]].sum(axis=1)
    result_final['green_other_ic']=result[['classification_forest_path',
                                    'classification_forest_road'
                                   ]].sum(axis=1)
    # detection
    result_final['bicycle_od']=result['detection_bicycle']
    result_final['bus_od']=result['detection_bus']
    result_final['car_od']=result['detection_car']
    result_final['motorcycle_od']=result['detection_motorcycle']
    result_final['person_od']=result['detection_person']
    result_final['traffic_light_od']=result['detection_traffic light']
    result_final['truck_od']=result['detection_truck']
    # edge dection
    result_final['canny_edge_llf']=result['edge_detection_edge_ratio']
    # blob detection
    result_final['no_of_blobs_llf']=result['blob_detection_blob_num']
    # hls
    result_final['hue_mean_llf']=result['hls_statistics_h_mean']
    result_final['hue_std_llf']=result['hls_statistics_h_mean']
    result_final['lightness_mean_llf']=result['hls_statistics_l_mean']
    result_final['lightness_std_llf']=result['hls_statistics_l_std']
    result_final['saturation_mean_llf']=result['hls_statistics_s_mean']
    result_final['saturation_std_llf']=result['hls_statistics_s_std']
    
    return result_final

result_final=featureEngineer(result,8)
print(result_final)

# # save the combined data
result_final.to_csv(os.path.join(root,'data/survey_result/survey_2nd_result_with_independent_variables.csv'))

## Predict perception
### Hyper parameter tuning

In [ ]:
root = "/home/ec2-user/SageMaker/"

# Global_vars
seed = 1234
num_folds = 10
n_jobs = 4
scoring='neg_root_mean_squared_error'
model_output_folder=os.path.join(root,'models/survey_2nd')
if not os.path.exists(model_output_folder):
    os.makedirs(model_output_folder)

# load the data
result=pd.read_csv(os.path.join(root,'data/survey_result/survey_2nd_result_with_independent_variables.csv'))

# create a list of target variables
target_variables=['beauty',
                  'building_attractiveness','cleanliness','cycling_attractiveness',
                  'living_attractiveness','safety','spaciousness'
                 ]
for target_variable in tqdm.tqdm(target_variables):
    X=result.iloc[:,9:]
    y = result[target_variable]
    # split X,y into train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)

    # Create pipelines
    pipelines = pipeline_utils.create_pipelines(seed)

    # Run same experiment with hypertuned parameters
    hypertuned_params = pipeline_utils.get_hypertune_params()
    hypertune_results_df = pipeline_utils.run_cv_and_test_hypertuned_params(X_train, y_train, X_test, y_test, 
                                                                            pipelines, scoring, seed, num_folds,
                                                                            hypertuned_params, target_variable, n_jobs, 
                                                                            model_output_folder)
    print(hypertune_results_df)

### Train models with hyper parameters

In [ ]:
root = "/home/ec2-user/SageMaker/"

# Global_vars
seed = 1234
n_jobs = 4
model_name='StandardScaler_LGBMR-PCA'
model_output_folder=os.path.join(root,'models/survey_2nd/train_models/')
if not os.path.exists(model_output_folder):
    os.makedirs(model_output_folder)

# load the data
result=pd.read_csv(os.path.join(root,'data/survey_result/survey_2nd_result_with_independent_variables.csv'))

# create a list of target variables
target_variables=['beauty',
                  'building_attractiveness','cleanliness','cycling_attractiveness',
                  'living_attractiveness','safety','spaciousness'
                 ]

# Lists for all the results
results = []
# go through the target variables
for target_variable in tqdm.tqdm(target_variables):
    # get X and y
    X=result.iloc[:,9:]
    y = result[target_variable]
    # split X,y into train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)
    
    # load the model to get hyper parameters
#     model= joblib.load(os.path.join(root,'models/survey_2nd',target_variable+"__"+model_name+'.joblib'))
#     hyper_params=model.best_params_
#     hyper_params_new={}
#     for key in hyper_params:
#         hyper_params_new[key.replace('LGBMR__','')] = hyper_params[key]
    
    hyper_params_new={
    "objective": "regression",
#     'num_leaves':100,
    "metric": "rmse",
    "verbosity": -1,
    "boosting_type": "gbdt"}
    # create a pipeline
    model_pipeline = Pipeline(steps=[
#                                      ('StandardScaler', StandardScaler()), 
#                                      ('PCA', PCA(0.9)),
                                     ('LGBMR', LGBMRegressor(**hyper_params_new,random_state=seed))
                                 ])
#     pipeline_temp =Pipeline(model_pipeline.steps[:-1])
#     X_trans = pipeline_temp.fit_transform(X_train,y_train)
#     eval_set = [(X_trans, y_train), (pipeline_temp.transform(X_test), y_test)]
    model_pipeline.fit(X_train,y_train,LGBMR__verbose=10,
                       LGBMR__early_stopping_rounds=35000,
                       LGBMR__eval_set = [(X_train, y_train), (X_test, y_test)])
    
    # predict 
    y_pred = model_pipeline.predict(X_test)
    # get different performance metrics
    score_mean_absolute_error=mean_absolute_error(y_test, y_pred)
    score_mean_absolute_percentage_error = mean_absolute_percentage_error(y_test, y_pred)
    score_root_mean_squared_error=mean_squared_error(y_test, y_pred, squared=False)
    score_r2=r2_score(y_test, y_pred)
    # create dict and append to rows_list
    results_dict = {"Target_variable": target_variable,
                    "mean_absolute_error":score_mean_absolute_error,
                    "mean_absolute_percentage_error":score_mean_absolute_percentage_error,
                    "root_mean_squared_error":score_root_mean_squared_error,
                    "r2_score":score_r2
                    }
    print(results_dict)
    results.append(results_dict)
    
    # plot the pred and actual value
    g=plt.scatter(y_test, y_pred, alpha=0.6)
    g.axes.set_xlabel('True Values ')
    g.axes.set_ylabel('Predictions ')
    title="Predictions vs. Actual Values for {}".format(target_variable)
    g.axes.set_title(title, fontdict={"fontsize": "25", "fontweight" : "3"})
    plt.savefig(os.path.join(root, 'output/perception_prediction/prediction_vs_actual_{}.jpg'.format(target_variable)),bbox_inches = "tight",dpi=400)
    plt.show()
    
    # save the model
    joblib.dump(model_pipeline, os.path.join(model_output_folder,target_variable+"__"+model_name+'.joblib'))
    
df = pd.DataFrame(results)
df.to_csv(os.path.join(model_output_folder, 'accuracy_result.csv'))

### predict for the rest of the data (including the ones surveyed)

In [ ]:
city_list=['Singapore',
           'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

model_output_folder=os.path.join(root,'models/survey_2nd/train_models/')
model_name='StandardScaler_LGBMR-PCA'

data_list=['segmentation',
           'detection',
           'classification',
           'edge_detection',
           'blob_detection',
           'hls_statistics'
          ]

# create a list of target variables
target_variables=['beauty',
                  'building_attractiveness','cleanliness','cycling_attractiveness',
                  'living_attractiveness','safety','spaciousness'
                 ]

for city in city_list:
    # load a dataframe of panoid
    sample_points=pd.read_csv(os.path.join(root,'data/meta_data_{}/{}_remaining_points.csv'.format(city,city)))
    sample_points=sample_points[['panoId']]
    sample_points=sample_points.rename(columns={'panoId':'pano_id'})
    sample_points['pano_id']='pano='+sample_points['pano_id']
    sample_points_keep=sample_points.copy()
    
    # go through data_list and left join the data to sample_points df
    for data in data_list:
        # load the data
        data_df=pd.read_csv(os.path.join(root,'data/tabular_data/{}/{}.csv'.format(city,data)))
        # left join the df_temp to result
        if not 'pano_id' in data_df.columns:
            data_df=data_df.rename(columns={data_df.columns[0]:'pano_id'})
        data_df.drop(data_df.filter(regex='Unnamed').columns, axis=1, inplace=True)
        data_df.columns=['pano_id' if col =='pano_id' else data+'_'+col for col in data_df.columns]
        sample_points=pd.merge(sample_points,data_df,
                              on='pano_id',how='left')
    result_final=featureEngineer(sample_points,1)
    # get X
    X=result_final.iloc[:,1:]
    print(X)
    # go through the target variables
    for target_variable in tqdm.tqdm(target_variables):
        # load the model
        model= joblib.load(os.path.join(model_output_folder,target_variable+"__"+model_name+'.joblib'))
        prediction=model.predict(X)
        prediction_df = pd.DataFrame(data=prediction, columns=["predicted_"+target_variable])
        sample_points_keep_copy=sample_points_keep.copy()
        concatenated_dataframes = pd.concat([sample_points_keep_copy, prediction_df], axis=1)
        print(concatenated_dataframes)
        
        # save the dataframe
        concatenated_dataframes.to_csv(os.path.join(root,'data/tabular_data/{}/predicted_{}.csv'.format(city,target_variable)))

### Replace predicted scores with the surveyed scores

# Combining Indicators
## non-SVI& SVI

## Connectivity
- Intersection with traffic lights
- Intersection without traffic lights
- Cul de sac

In [2]:
city_list=['Singapore',
           'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# min-max scaler
scaler = MinMaxScaler()

data_list=['intersections_with_traffic_lights',
           'intersections_without_traffic_lights',
           'cul_de_sac'
          ]

for city in city_list:
    # load location
    location=os.path.join(root,'data/meta_data_{}/{}_remaining_points.csv'.format(city,city))
    location_df=pd.read_csv(location)
    location_df['panoId']='pano='+location_df['panoId']
    location_df=location_df.rename(columns={'panoId':'pano_id'})
    
   # go through data_list and left join it to location_df
    for data in data_list:
        # import as df
        data_df=pd.read_csv(os.path.join(root,'data/tabular_data/{}/{}.csv'.format(city,data)))
        data_df=data_df.iloc[:,1:]
        data_df=data_df.rename(columns={data_df.columns[0]:'pano_id',data_df.columns[-1]:data})
    
        # merge and scale them
        location_df=pd.merge(location_df,
                         data_df,
                         on='pano_id',
                         how='left'
                        ) 
        location_df.to_csv(os.path.join(root,'data/tabular_data/{}/connectivity_before_scaling.csv'.format(city)))
        
# scaling
# put both cities data together first
df_temp=pd.DataFrame()
for city in city_list:
    connectivity_df=pd.read_csv(os.path.join(root,'data/tabular_data/{}/connectivity_before_scaling.csv'.format(city)))
    connectivity_df['city']=city
    df_temp=df_temp.append(connectivity_df)
# scaling using Min-Max scale
cols_transfomation_negative=data_list
# fit the scaler
scaler = MinMaxScaler()
df_temp[cols_transfomation_negative]=scaler.fit_transform(df_temp[cols_transfomation_negative])
df_temp[cols_transfomation_negative]=1-df_temp[cols_transfomation_negative]
# calculate the indicator
df_temp['connectivity']=df_temp[data_list].sum(axis=1)*(20/len(data_list))

# save seperately by cities
for city in city_list:
    df_temp_city=df_temp.loc[df_temp['city']==city]
    print(df_temp_city)
    df_temp_city.to_csv(os.path.join(root,'data/tabular_data/{}/connectivity.csv'.format(city)))

      Unnamed: 0  Unnamed: 0.1  \
0              0             0   
1              1             1   
2              2             2   
3              3             3   
4              4             4   
...          ...           ...   
5828        5828          6115   
5829        5829          6116   
5830        5830          6117   
5831        5831          6119   
5832        5832          6120   

                                                pano_id     panoLon   panoLat  \
0                           pano=EIzSniBOXDgT4m2fGH2SNQ  103.914512  1.400917   
1                           pano=tzvazBY0ag5RgAsvJgHvZA  103.965906  1.375535   
2                           pano=FJjCz3jIAWYYnGq4PvR9HA  103.819038  1.293706   
3                           pano=TfQN73pv-6mAZy2QnMIAPA  103.703747  1.341976   
4                           pano=PATYQYa4O43_LjJ_JMbrFw  103.900929  1.311461   
...                                                 ...         ...       ...   
5828  pano=CAoSLEFGMVFpc

## Environment
- Slope
- Number of points of interest
- Shannon land use mix index
- Air quality index (AQI)
- Scenery: buildings
- Scenery: greenery
- Scenery: water

In [3]:
city_list=['Singapore',
           'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# min-max scaler
scaler = MinMaxScaler()

data_list=['slope',
           'poi',
           'land_use',
           'aqi',
           'scenery_building',
           'scenery_greenery',
           'scenery_water'
          ]

for city in city_list:
    # load location
    location=os.path.join(root,'data/meta_data_{}/{}_remaining_points.csv'.format(city,city))
    location_df=pd.read_csv(location)
    location_df['panoId']='pano='+location_df['panoId']
    location_df=location_df.rename(columns={'panoId':'pano_id'})
    
    # go through data_list and left join it to location_df
    for data in data_list:
        # import as df
        data_df=pd.read_csv(os.path.join(root,'data/tabular_data/{}/{}.csv'.format(city,data)))
        data_df=data_df.iloc[:,1:]
        data_df=data_df.rename(columns={data_df.columns[0]:'pano_id',data_df.columns[-1]:data})
    
        # merge and scale them
        location_df=pd.merge(location_df,
                         data_df,
                         on='pano_id',
                         how='left'
                        ) 
        location_df.to_csv(os.path.join(root,'data/tabular_data/{}/environment_before_scaling.csv'.format(city)))
        
# scaling
# put both cities data together first
df_temp=pd.DataFrame()
for city in city_list:
    environment_df=pd.read_csv(os.path.join(root,'data/tabular_data/{}/environment_before_scaling.csv'.format(city)))
    environment_df['city']=city
    df_temp=df_temp.append(environment_df)
# scaling using Min-Max scale
cols_transfomation_positive=['poi']
cols_transfomation_negative=['slope','aqi']
# fit the scaler
scaler = MinMaxScaler()
df_temp[cols_transfomation_positive]=scaler.fit_transform(df_temp[cols_transfomation_positive])
df_temp[cols_transfomation_negative]=scaler.fit_transform(df_temp[cols_transfomation_negative])
df_temp[cols_transfomation_negative]=1-df_temp[cols_transfomation_negative]

# calculate the indicator
df_temp['environment']=df_temp[data_list].sum(axis=1)*(20/len(data_list))
# save seperately by cities
for city in city_list:
    df_temp_city=df_temp.loc[df_temp['city']==city]
    print(df_temp_city)
    df_temp_city.to_csv(os.path.join(root,'data/tabular_data/{}/environment.csv'.format(city)))

      Unnamed: 0  Unnamed: 0.1  \
0              0             0   
1              1             1   
2              2             2   
3              3             3   
4              4             4   
...          ...           ...   
5828        5828          6115   
5829        5829          6116   
5830        5830          6117   
5831        5831          6119   
5832        5832          6120   

                                                pano_id     panoLon   panoLat  \
0                           pano=EIzSniBOXDgT4m2fGH2SNQ  103.914512  1.400917   
1                           pano=tzvazBY0ag5RgAsvJgHvZA  103.965906  1.375535   
2                           pano=FJjCz3jIAWYYnGq4PvR9HA  103.819038  1.293706   
3                           pano=TfQN73pv-6mAZy2QnMIAPA  103.703747  1.341976   
4                           pano=PATYQYa4O43_LjJ_JMbrFw  103.900929  1.311461   
...                                                 ...         ...       ...   
5828  pano=CAoSLEFGMVFpc

## Infrastructure
- Type of road
- Presence of potholes
- Presence of street light
- Presence of bike lanes
- Number of transit facilities
- Type of pavement (paved vs unpaved)
- Presence of street amenities (e.g., trash cans and benches)
- Presence of utility pole
- Presence of bike parking
- Road width
- Presence of sidewalk
- Presense and quality of crosswalk (with/without traffic lights, traffic signs)
- Presence of curb cuts

In [4]:
city_list=['Singapore',
           'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# min-max scaler
scaler = MinMaxScaler()

data_list=['road_type',
           'pavement',
           'street_light',
           'bike_lanes',
           'transit',
           'surface_type',
           'street_amenity',
           'utility_pole',
           'bike_parking',
           'road_width',
           'side_walk',
           'cross_walk',
           'accessibility'
          ]

for city in city_list:
    # load location
    location=os.path.join(root,'data/meta_data_{}/{}_remaining_points.csv'.format(city,city))
    location_df=pd.read_csv(location)
    location_df['panoId']='pano='+location_df['panoId']
    location_df=location_df.rename(columns={'panoId':'pano_id'})
    
    # go through data_list and left join it to location_df
    for data in data_list:
        # import as df
        data_df=pd.read_csv(os.path.join(root,'data/tabular_data/{}/{}.csv'.format(city,data)))
        data_df=data_df.iloc[:,1:]
        data_df=data_df.rename(columns={data_df.columns[0]:'pano_id',data_df.columns[-1]:data})
    
        # merge and scale them
        location_df=pd.merge(location_df,
                         data_df,
                         on='pano_id',
                         how='left'
                        ) 
        location_df.to_csv(os.path.join(root,'data/tabular_data/{}/infrastructure_before_scaling.csv'.format(city)))
        
# scaling
# put both cities data together first
df_temp=pd.DataFrame()
for city in city_list:
    infrastructure_df=pd.read_csv(os.path.join(root,'data/tabular_data/{}/infrastructure_before_scaling.csv'.format(city)))
    infrastructure_df['city']=city
    df_temp=df_temp.append(infrastructure_df)
# scaling using Min-Max scale
cols_transfomation_positive=['transit']
# fit the scaler
scaler = MinMaxScaler()
df_temp[cols_transfomation_positive]=scaler.fit_transform(df_temp[cols_transfomation_positive])
# 1 if >0
cols_transfomation_above_0=['street_light','bike_lanes','street_amenity','bike_parking',
                            'side_walk','cross_walk','accessibility']
for col in cols_transfomation_above_0:
    cond_list=[df_temp[col]>0,df_temp[col]==0]
    choice_list=[1,0]
    df_temp[col]=np.select(cond_list,choice_list)
# 1 if ==0
cols_transfomation_equal_to_0=['pavement','utility_pole']
for col in cols_transfomation_equal_to_0:
    cond_list=[df_temp[col]==0,df_temp[col]>0]
    choice_list=[1,0]
    df_temp[col]=np.select(cond_list,choice_list)

# calculate the indicator
df_temp['infrastructure']=df_temp[data_list].sum(axis=1)*(20/len(data_list))
# save seperately by cities
for city in city_list:
    df_temp_city=df_temp.loc[df_temp['city']==city]
    print(df_temp_city)
    df_temp_city.to_csv(os.path.join(root,'data/tabular_data/{}/infrastructure.csv'.format(city)))

      Unnamed: 0  Unnamed: 0.1  \
0              0             0   
1              1             1   
2              2             2   
3              3             3   
4              4             4   
...          ...           ...   
5828        5828          6115   
5829        5829          6116   
5830        5830          6117   
5831        5831          6119   
5832        5832          6120   

                                                pano_id     panoLon   panoLat  \
0                           pano=EIzSniBOXDgT4m2fGH2SNQ  103.914512  1.400917   
1                           pano=tzvazBY0ag5RgAsvJgHvZA  103.965906  1.375535   
2                           pano=FJjCz3jIAWYYnGq4PvR9HA  103.819038  1.293706   
3                           pano=TfQN73pv-6mAZy2QnMIAPA  103.703747  1.341976   
4                           pano=PATYQYa4O43_LjJ_JMbrFw  103.900929  1.311461   
...                                                 ...         ...       ...   
5828  pano=CAoSLEFGMVFpc

## Vehicle_Cyclist_Interaction
- No. of vehicles
- Presence of off-street parking lot spaces
- Number of speed bumps / choker / roundabout
- traffic light / stop sign

In [5]:
city_list=['Singapore',
           'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# min-max scaler
scaler = MinMaxScaler()

data_list=['no_of_vehicles',
           'street_parking',
           'traffic_calming',
           'traffic_light_stop_sign'
          ]

for city in city_list:
    # load location
    location=os.path.join(root,'data/meta_data_{}/{}_remaining_points.csv'.format(city,city))
    location_df=pd.read_csv(location)
    location_df['panoId']='pano='+location_df['panoId']
    location_df=location_df.rename(columns={'panoId':'pano_id'})
    
    # go through data_list and left join it to location_df
    for data in data_list:
        # import as df
        data_df=pd.read_csv(os.path.join(root,'data/tabular_data/{}/{}.csv'.format(city,data)))
        data_df=data_df.iloc[:,1:]
        data_df=data_df.rename(columns={data_df.columns[0]:'pano_id',data_df.columns[-1]:data})
    
        # merge and scale them
        location_df=pd.merge(location_df,
                         data_df,
                         on='pano_id',
                         how='left'
                        ) 
        location_df.to_csv(os.path.join(root,'data/tabular_data/{}/vehicle_cyclist_interaction_before_scaling.csv'.format(city)))
        
# scaling
# put both cities data together first
df_temp=pd.DataFrame()
for city in city_list:
    vehicle_cyclist_interaction_df=pd.read_csv(os.path.join(root,'data/tabular_data/{}/vehicle_cyclist_interaction_before_scaling.csv'.format(city)))
    vehicle_cyclist_interaction_df['city']=city
    df_temp=df_temp.append(vehicle_cyclist_interaction_df)

# scaling using Min-Max scale
cols_transfomation_negative=['no_of_vehicles']
df_temp[cols_transfomation_negative]=scaler.fit_transform(df_temp[cols_transfomation_negative])
df_temp[cols_transfomation_negative]=1-df_temp[cols_transfomation_negative]

# 1 if >0
cols_transfomation_above_0=['traffic_calming']
for col in cols_transfomation_above_0:
    cond_list=[df_temp[col]>0]
    choice_list=[1]
    df_temp[col]=np.select(cond_list,choice_list)

# calculate the indicator
df_temp['vehicle_cyclist_interaction']=df_temp[data_list].sum(axis=1)*(20/len(data_list))
# save seperately by cities
for city in city_list:
    df_temp_city=df_temp.loc[df_temp['city']==city]
    print(df_temp_city.describe())
    df_temp_city.to_csv(os.path.join(root,'data/tabular_data/{}/vehicle_cyclist_interaction.csv'.format(city)))

        Unnamed: 0  Unnamed: 0.1      panoLon      panoLat      distDiff  \
count  5833.000000   5833.000000  5833.000000  5833.000000  5.833000e+03   
mean   2916.000000   3061.086233   103.827390     1.347023  7.049910e-05   
std    1683.986391   1764.040159     0.084191     0.044801  1.839192e-04   
min       0.000000      0.000000   103.611153     1.244036  3.210881e-07   
25%    1458.000000   1537.000000   103.764845     1.313970  2.542029e-05   
50%    2916.000000   3060.000000   103.837668     1.339950  4.351301e-05   
75%    4374.000000   4583.000000   103.889764     1.375763  7.797647e-05   
max    5832.000000   6120.000000   104.025514     1.469106  8.622213e-03   

       no_of_vehicles         length  street_parking_score  street_parking  \
count     5833.000000    5833.000000           5833.000000     5833.000000   
mean         0.881842    5097.670675           5072.027550        0.994098   
std          0.136240    4607.236936           4602.713191        0.021811   
min

## Perception
- beauty
- building_attractiveness
- cleanliness
-  cycling_attractiveness
- living_attractiveness
- safety
- spaciousness

In [6]:
city_list=['Singapore',
           'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"


data_list=['beauty',
          'building_attractiveness','cleanliness','cycling_attractiveness',
          'living_attractiveness','safety','spaciousness'
          ]

for city in city_list:
    # load location
    location=os.path.join(root,'data/meta_data_{}/{}_remaining_points.csv'.format(city,city))
    location_df=pd.read_csv(location)
    location_df['panoId']='pano='+location_df['panoId']
    location_df=location_df.rename(columns={'panoId':'pano_id'})
    
    # go through data_list and left join it to location_df
    for data in data_list:
        # import as df
        data_df=pd.read_csv(os.path.join(root,'data/tabular_data/{}/predicted_{}.csv'.format(city,data)))
        data_df=data_df.iloc[:,1:]
        # merge and scale them
        location_df=pd.merge(location_df,
                         data_df,
                         on='pano_id',
                         how='left'
                        )
    # scaling
    location_df = location_df.apply(lambda x: x/10 if x.name.startswith('predicted_') else x)
    location_df['perception']=location_df[location_df.columns[pd.Series(location_df.columns).str.startswith('predicted_')]].\
        sum(axis=1)*(20/len(data_list))
    print(location_df)
    location_df.to_csv(os.path.join(root,'data/tabular_data/{}/perception.csv'.format(city)))

      Unnamed: 0                                            pano_id  \
0              0                        pano=EIzSniBOXDgT4m2fGH2SNQ   
1              1                        pano=tzvazBY0ag5RgAsvJgHvZA   
2              2                        pano=FJjCz3jIAWYYnGq4PvR9HA   
3              3                        pano=TfQN73pv-6mAZy2QnMIAPA   
4              4                        pano=PATYQYa4O43_LjJ_JMbrFw   
...          ...                                                ...   
5828        6115  pano=CAoSLEFGMVFpcE9YcGFGMVpBOURjN21NS0NQS1h3L...   
5829        6116                        pano=4I7X92FmvGD7ALNXMZQuAg   
5830        6117                        pano=EghtrEJcRO-l7XlA21zaNQ   
5831        6119                        pano=O6UhfdrDWVrKDmJTaOLmFA   
5832        6120                        pano=uvufId4xvxyRl-4TUehx_A   

         panoLon   panoLat  distDiff  predicted_beauty  \
0     103.914512  1.400917  0.000084          0.677530   
1     103.965906  1.375535  0.0

## Bikeability
- connectivity
- environment
- infrastructure
- vehicle cyclist interaction
- perception

In [7]:
city_list=['Singapore',
           'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"


data_list=['connectivity',
           'environment',
           'infrastructure',
           'vehicle_cyclist_interaction',
           'perception'
          ]

for city in city_list:
    # load location
    location=os.path.join(root,'data/meta_data_{}/{}_remaining_points.csv'.format(city,city))
    location_df=pd.read_csv(location)
    location_df['panoId']='pano='+location_df['panoId']
    location_df=location_df.rename(columns={'panoId':'pano_id'})
    
    # go through data_list and left join it to location_df
    for data in data_list:
        # import as df
        data_df=pd.read_csv(os.path.join(root,'data/tabular_data/{}/{}.csv'.format(city,data)))
        data_df=data_df.iloc[:,1:]
        # merge and scale them
        location_df=pd.merge(location_df,
                         data_df[['pano_id',data]],
                         on='pano_id',
                         how='left'
                        )
        location_df.drop(location_df.filter(regex='Unnamed').columns, axis=1, inplace=True)
        
    # scaling
    location_df['bikeability']=location_df.iloc[:,-5:].sum(axis=1)
    print(location_df)
    location_df.to_csv(os.path.join(root,'data/tabular_data/{}/bikeability.csv'.format(city)))

                                                pano_id     panoLon   panoLat  \
0                           pano=EIzSniBOXDgT4m2fGH2SNQ  103.914512  1.400917   
1                           pano=tzvazBY0ag5RgAsvJgHvZA  103.965906  1.375535   
2                           pano=FJjCz3jIAWYYnGq4PvR9HA  103.819038  1.293706   
3                           pano=TfQN73pv-6mAZy2QnMIAPA  103.703747  1.341976   
4                           pano=PATYQYa4O43_LjJ_JMbrFw  103.900929  1.311461   
...                                                 ...         ...       ...   
5828  pano=CAoSLEFGMVFpcE9YcGFGMVpBOURjN21NS0NQS1h3L...  103.835671  1.289808   
5829                        pano=4I7X92FmvGD7ALNXMZQuAg  103.966108  1.373809   
5830                        pano=EghtrEJcRO-l7XlA21zaNQ  103.701994  1.344878   
5831                        pano=O6UhfdrDWVrKDmJTaOLmFA  103.632836  1.324383   
5832                        pano=uvufId4xvxyRl-4TUehx_A  103.747407  1.428744   

      distDiff  connectivit

## only Non-SVI

### Connectivity
- Intersection with traffic lights
- Intersection without traffic lights
- Cul de sac

In [8]:
num_category=4
city_list=['Singapore',
           'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# min-max scaler
scaler = MinMaxScaler()

data_list=['intersections_with_traffic_lights',
           'intersections_without_traffic_lights',
           'cul_de_sac'
          ]

for city in city_list:
    # load location
    location=os.path.join(root,'data/meta_data_{}/{}_remaining_points.csv'.format(city,city))
    location_df=pd.read_csv(location)
    location_df['panoId']='pano='+location_df['panoId']
    location_df=location_df.rename(columns={'panoId':'pano_id'})
    
   # go through data_list and left join it to location_df
    for data in data_list:
        # import as df
        data_df=pd.read_csv(os.path.join(root,'data/tabular_data/{}/{}.csv'.format(city,data)))
        data_df=data_df.iloc[:,1:]
        data_df=data_df.rename(columns={data_df.columns[0]:'pano_id',data_df.columns[-1]:data})
    
        # merge and scale them
        location_df=pd.merge(location_df,
                         data_df,
                         on='pano_id',
                         how='left'
                        ) 
        location_df.to_csv(os.path.join(root,'data/tabular_data/{}/only_non_svi_connectivity_before_scaling.csv'.format(city)))
        
# scaling
# put both cities data together first
df_temp=pd.DataFrame()
for city in city_list:
    connectivity_df=pd.read_csv(os.path.join(root,'data/tabular_data/{}/only_non_svi_connectivity_before_scaling.csv'.format(city)))
    connectivity_df['city']=city
    df_temp=df_temp.append(connectivity_df)
# scaling using Min-Max scale
cols_transfomation_negative=data_list
# fit the scaler
scaler = MinMaxScaler()
df_temp[cols_transfomation_negative]=scaler.fit_transform(df_temp[cols_transfomation_negative])
df_temp[cols_transfomation_negative]=1-df_temp[cols_transfomation_negative]
# calculate the indicator
df_temp['connectivity']=df_temp[data_list].sum(axis=1)*((100/num_category)/len(data_list))

# save seperately by cities
for city in city_list:
    df_temp_city=df_temp.loc[df_temp['city']==city]
    print(df_temp_city)
    df_temp_city.to_csv(os.path.join(root,'data/tabular_data/{}/only_non_svi_connectivity.csv'.format(city)))

      Unnamed: 0  Unnamed: 0.1  \
0              0             0   
1              1             1   
2              2             2   
3              3             3   
4              4             4   
...          ...           ...   
5828        5828          6115   
5829        5829          6116   
5830        5830          6117   
5831        5831          6119   
5832        5832          6120   

                                                pano_id     panoLon   panoLat  \
0                           pano=EIzSniBOXDgT4m2fGH2SNQ  103.914512  1.400917   
1                           pano=tzvazBY0ag5RgAsvJgHvZA  103.965906  1.375535   
2                           pano=FJjCz3jIAWYYnGq4PvR9HA  103.819038  1.293706   
3                           pano=TfQN73pv-6mAZy2QnMIAPA  103.703747  1.341976   
4                           pano=PATYQYa4O43_LjJ_JMbrFw  103.900929  1.311461   
...                                                 ...         ...       ...   
5828  pano=CAoSLEFGMVFpc

### Environment
- Slope
- Number of points of interest
- Shannon land use mix index
- Air quality index (AQI)

In [9]:
num_category=4
city_list=['Singapore',
           'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# min-max scaler
scaler = MinMaxScaler()

data_list=['slope',
           'poi',
           'land_use',
           'aqi'
          ]

for city in city_list:
    # load location
    location=os.path.join(root,'data/meta_data_{}/{}_remaining_points.csv'.format(city,city))
    location_df=pd.read_csv(location)
    location_df['panoId']='pano='+location_df['panoId']
    location_df=location_df.rename(columns={'panoId':'pano_id'})
    
    # go through data_list and left join it to location_df
    for data in data_list:
        # import as df
        data_df=pd.read_csv(os.path.join(root,'data/tabular_data/{}/{}.csv'.format(city,data)))
        data_df=data_df.iloc[:,1:]
        data_df=data_df.rename(columns={data_df.columns[0]:'pano_id',data_df.columns[-1]:data})
    
        # merge and scale them
        location_df=pd.merge(location_df,
                         data_df,
                         on='pano_id',
                         how='left'
                        ) 
        location_df.to_csv(os.path.join(root,'data/tabular_data/{}/only_non_svi_environment_before_scaling.csv'.format(city)))
        
# scaling
# put both cities data together first
df_temp=pd.DataFrame()
for city in city_list:
    environment_df=pd.read_csv(os.path.join(root,'data/tabular_data/{}/only_non_svi_environment_before_scaling.csv'.format(city)))
    environment_df['city']=city
    df_temp=df_temp.append(environment_df)
# scaling using Min-Max scale
cols_transfomation_positive=['poi']
cols_transfomation_negative=['slope','aqi']
# fit the scaler
scaler = MinMaxScaler()
df_temp[cols_transfomation_positive]=scaler.fit_transform(df_temp[cols_transfomation_positive])
df_temp[cols_transfomation_negative]=scaler.fit_transform(df_temp[cols_transfomation_negative])
df_temp[cols_transfomation_negative]=1-df_temp[cols_transfomation_negative]

# calculate the indicator
df_temp['environment']=df_temp[data_list].sum(axis=1)*((100/num_category)/len(data_list))
# save seperately by cities
for city in city_list:
    df_temp_city=df_temp.loc[df_temp['city']==city]
    print(df_temp_city)
    df_temp_city.to_csv(os.path.join(root,'data/tabular_data/{}/only_non_svi_environment.csv'.format(city)))

      Unnamed: 0  Unnamed: 0.1  \
0              0             0   
1              1             1   
2              2             2   
3              3             3   
4              4             4   
...          ...           ...   
5828        5828          6115   
5829        5829          6116   
5830        5830          6117   
5831        5831          6119   
5832        5832          6120   

                                                pano_id     panoLon   panoLat  \
0                           pano=EIzSniBOXDgT4m2fGH2SNQ  103.914512  1.400917   
1                           pano=tzvazBY0ag5RgAsvJgHvZA  103.965906  1.375535   
2                           pano=FJjCz3jIAWYYnGq4PvR9HA  103.819038  1.293706   
3                           pano=TfQN73pv-6mAZy2QnMIAPA  103.703747  1.341976   
4                           pano=PATYQYa4O43_LjJ_JMbrFw  103.900929  1.311461   
...                                                 ...         ...       ...   
5828  pano=CAoSLEFGMVFpc

### Infrastructure
- Type of road
- Number of transit facilities
- Type of pavement (paved vs unpaved)
- Road width

In [10]:
num_category=4
city_list=['Singapore',
           'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# min-max scaler
scaler = MinMaxScaler()

data_list=['road_type',
           'transit',
           'surface_type',
           'road_width'
          ]

for city in city_list:
    # load location
    location=os.path.join(root,'data/meta_data_{}/{}_remaining_points.csv'.format(city,city))
    location_df=pd.read_csv(location)
    location_df['panoId']='pano='+location_df['panoId']
    location_df=location_df.rename(columns={'panoId':'pano_id'})
    
    # go through data_list and left join it to location_df
    for data in data_list:
        # import as df
        data_df=pd.read_csv(os.path.join(root,'data/tabular_data/{}/{}.csv'.format(city,data)))
        data_df=data_df.iloc[:,1:]
        data_df=data_df.rename(columns={data_df.columns[0]:'pano_id',data_df.columns[-1]:data})
    
        # merge and scale them
        location_df=pd.merge(location_df,
                         data_df,
                         on='pano_id',
                         how='left'
                        ) 
        location_df.to_csv(os.path.join(root,'data/tabular_data/{}/only_non_svi_infrastructure_before_scaling.csv'.format(city)))
        
# scaling
# put both cities data together first
df_temp=pd.DataFrame()
for city in city_list:
    infrastructure_df=pd.read_csv(os.path.join(root,'data/tabular_data/{}/only_non_svi_infrastructure_before_scaling.csv'.format(city)))
    infrastructure_df['city']=city
    df_temp=df_temp.append(infrastructure_df)
# scaling using Min-Max scale
cols_transfomation_positive=['transit']
# fit the scaler
scaler = MinMaxScaler()
df_temp[cols_transfomation_positive]=scaler.fit_transform(df_temp[cols_transfomation_positive])

# calculate the indicator
df_temp['infrastructure']=df_temp[data_list].sum(axis=1)*((100/num_category)/len(data_list))
# save seperately by cities
for city in city_list:
    df_temp_city=df_temp.loc[df_temp['city']==city]
    print(df_temp_city.describe())
    df_temp_city.to_csv(os.path.join(root,'data/tabular_data/{}/only_non_svi_infrastructure.csv'.format(city)))

        Unnamed: 0  Unnamed: 0.1      panoLon      panoLat      distDiff  \
count  5833.000000   5833.000000  5833.000000  5833.000000  5.833000e+03   
mean   2916.000000   3061.086233   103.827390     1.347023  7.049910e-05   
std    1683.986391   1764.040159     0.084191     0.044801  1.839192e-04   
min       0.000000      0.000000   103.611153     1.244036  3.210881e-07   
25%    1458.000000   1537.000000   103.764845     1.313970  2.542029e-05   
50%    2916.000000   3060.000000   103.837668     1.339950  4.351301e-05   
75%    4374.000000   4583.000000   103.889764     1.375763  7.797647e-05   
max    5832.000000   6120.000000   104.025514     1.469106  8.622213e-03   

           length_x  road_type_score    road_type      transit      length_y  \
count   5833.000000      5833.000000  5833.000000  5833.000000   5833.000000   
mean    2790.531982      1442.120853     0.526952     0.066832   1477.746906   
std     1365.407053       751.281260     0.153542     0.105506   1985.84524

### Vehicle_Cyclist_Interaction
- Presence of off-street parking lot spaces
- Number of speed bumps / choker / roundabout

In [11]:
num_category=4
city_list=['Singapore',
           'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# min-max scaler
scaler = MinMaxScaler()

data_list=[
           'street_parking',
           'traffic_calming'
          ]

for city in city_list:
    # load location
    location=os.path.join(root,'data/meta_data_{}/{}_remaining_points.csv'.format(city,city))
    location_df=pd.read_csv(location)
    location_df['panoId']='pano='+location_df['panoId']
    location_df=location_df.rename(columns={'panoId':'pano_id'})
    
    # go through data_list and left join it to location_df
    for data in data_list:
        # import as df
        data_df=pd.read_csv(os.path.join(root,'data/tabular_data/{}/{}.csv'.format(city,data)))
        data_df=data_df.iloc[:,1:]
        data_df=data_df.rename(columns={data_df.columns[0]:'pano_id',data_df.columns[-1]:data})
    
        # merge and scale them
        location_df=pd.merge(location_df,
                         data_df,
                         on='pano_id',
                         how='left'
                        ) 
        location_df.to_csv(os.path.join(root,'data/tabular_data/{}/only_non_svi_vehicle_cyclist_interaction_before_scaling.csv'.format(city)))
        
# scaling
# put both cities data together first
df_temp=pd.DataFrame()
for city in city_list:
    vehicle_cyclist_interaction_df=pd.read_csv(os.path.join(root,'data/tabular_data/{}/only_non_svi_vehicle_cyclist_interaction_before_scaling.csv'.format(city)))
    vehicle_cyclist_interaction_df['city']=city
    df_temp=df_temp.append(vehicle_cyclist_interaction_df)

# 1 if >0
cols_transfomation_above_0=['traffic_calming']
for col in cols_transfomation_above_0:
    cond_list=[df_temp[col]>0,df_temp[col]==0]
    choice_list=[1,0]
    df_temp[col]=np.select(cond_list,choice_list)

# calculate the indicator
df_temp['vehicle_cyclist_interaction']=df_temp[data_list].sum(axis=1)*((100/num_category)/len(data_list))
# save seperately by cities
for city in city_list:
    df_temp_city=df_temp.loc[df_temp['city']==city]
    print(df_temp_city)
    df_temp_city.to_csv(os.path.join(root,'data/tabular_data/{}/only_non_svi_vehicle_cyclist_interaction.csv'.format(city)))

      Unnamed: 0  Unnamed: 0.1  \
0              0             0   
1              1             1   
2              2             2   
3              3             3   
4              4             4   
...          ...           ...   
5828        5828          6115   
5829        5829          6116   
5830        5830          6117   
5831        5831          6119   
5832        5832          6120   

                                                pano_id     panoLon   panoLat  \
0                           pano=EIzSniBOXDgT4m2fGH2SNQ  103.914512  1.400917   
1                           pano=tzvazBY0ag5RgAsvJgHvZA  103.965906  1.375535   
2                           pano=FJjCz3jIAWYYnGq4PvR9HA  103.819038  1.293706   
3                           pano=TfQN73pv-6mAZy2QnMIAPA  103.703747  1.341976   
4                           pano=PATYQYa4O43_LjJ_JMbrFw  103.900929  1.311461   
...                                                 ...         ...       ...   
5828  pano=CAoSLEFGMVFpc

### Bikeability
- connectivity
- environment
- infrastructure
- vehicle cyclist interaction

In [12]:
num_category=4
city_list=['Singapore',
           'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"


data_list=['connectivity',
           'environment',
           'infrastructure',
           'vehicle_cyclist_interaction'
          ]

for city in city_list:
    # load location
    location=os.path.join(root,'data/meta_data_{}/{}_remaining_points.csv'.format(city,city))
    location_df=pd.read_csv(location)
    location_df['panoId']='pano='+location_df['panoId']
    location_df=location_df.rename(columns={'panoId':'pano_id'})
    
    # go through data_list and left join it to location_df
    for data in data_list:
        # import as df
        data_df=pd.read_csv(os.path.join(root,'data/tabular_data/{}/only_non_svi_{}.csv'.format(city,data)))
        data_df=data_df.iloc[:,1:]
        # merge and scale them
        location_df=pd.merge(location_df,
                         data_df[['pano_id',data]],
                         on='pano_id',
                         how='left'
                        )
        location_df.drop(location_df.filter(regex='Unnamed').columns, axis=1, inplace=True)
        
    # scaling
    location_df['bikeability']=location_df.iloc[:,-(num_category):].sum(axis=1)
    print(location_df)
    location_df.to_csv(os.path.join(root,'data/tabular_data/{}/only_non_svi_bikeability.csv'.format(city)))

                                                pano_id     panoLon   panoLat  \
0                           pano=EIzSniBOXDgT4m2fGH2SNQ  103.914512  1.400917   
1                           pano=tzvazBY0ag5RgAsvJgHvZA  103.965906  1.375535   
2                           pano=FJjCz3jIAWYYnGq4PvR9HA  103.819038  1.293706   
3                           pano=TfQN73pv-6mAZy2QnMIAPA  103.703747  1.341976   
4                           pano=PATYQYa4O43_LjJ_JMbrFw  103.900929  1.311461   
...                                                 ...         ...       ...   
5828  pano=CAoSLEFGMVFpcE9YcGFGMVpBOURjN21NS0NQS1h3L...  103.835671  1.289808   
5829                        pano=4I7X92FmvGD7ALNXMZQuAg  103.966108  1.373809   
5830                        pano=EghtrEJcRO-l7XlA21zaNQ  103.701994  1.344878   
5831                        pano=O6UhfdrDWVrKDmJTaOLmFA  103.632836  1.324383   
5832                        pano=uvufId4xvxyRl-4TUehx_A  103.747407  1.428744   

      distDiff  connectivit

## only SVI

### Environment
- Scenery: buildings
- Scenery: greenery
- Scenery: water

In [13]:
num_category=4
city_list=['Singapore',
           'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# min-max scaler
scaler = MinMaxScaler()

data_list=['scenery_building',
           'scenery_greenery',
           'scenery_water'
          ]

for city in city_list:
    # load location
    location=os.path.join(root,'data/meta_data_{}/{}_remaining_points.csv'.format(city,city))
    location_df=pd.read_csv(location)
    location_df['panoId']='pano='+location_df['panoId']
    location_df=location_df.rename(columns={'panoId':'pano_id'})
    
    # go through data_list and left join it to location_df
    for data in data_list:
        # import as df
        data_df=pd.read_csv(os.path.join(root,'data/tabular_data/{}/{}.csv'.format(city,data)))
        data_df=data_df.iloc[:,1:]
        data_df=data_df.rename(columns={data_df.columns[0]:'pano_id',data_df.columns[-1]:data})
    
        # merge and scale them
        location_df=pd.merge(location_df,
                         data_df,
                         on='pano_id',
                         how='left'
                        ) 
        location_df.to_csv(os.path.join(root,'data/tabular_data/{}/only_svi_environment_before_scaling.csv'.format(city)))
        

    # calculate the indicator
    location_df['environment']=location_df[data_list].sum(axis=1)*((100/num_category)/len(data_list))
    print(location_df)
    location_df.to_csv(os.path.join(root,'data/tabular_data/{}/only_svi_environment.csv'.format(city)))

      Unnamed: 0                                            pano_id  \
0              0                        pano=EIzSniBOXDgT4m2fGH2SNQ   
1              1                        pano=tzvazBY0ag5RgAsvJgHvZA   
2              2                        pano=FJjCz3jIAWYYnGq4PvR9HA   
3              3                        pano=TfQN73pv-6mAZy2QnMIAPA   
4              4                        pano=PATYQYa4O43_LjJ_JMbrFw   
...          ...                                                ...   
5828        6115  pano=CAoSLEFGMVFpcE9YcGFGMVpBOURjN21NS0NQS1h3L...   
5829        6116                        pano=4I7X92FmvGD7ALNXMZQuAg   
5830        6117                        pano=EghtrEJcRO-l7XlA21zaNQ   
5831        6119                        pano=O6UhfdrDWVrKDmJTaOLmFA   
5832        6120                        pano=uvufId4xvxyRl-4TUehx_A   

         panoLon   panoLat  distDiff  construction--structure--building  \
0     103.914512  1.400917  0.000084                           0.084285 

## Infrastructure
- Presence of potholes
- Presence of street light
- Presence of bike lanes
- Presence of street amenities (e.g., trash cans and benches)
- Presence of utility pole
- Presence of bike parking
- Presence of sidewalk
- Presense and quality of crosswalk (with/without traffic lights, traffic signs)
- Presence of curb cuts

In [14]:
num_category=4
city_list=['Singapore',
           'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# min-max scaler
scaler = MinMaxScaler()

data_list=[
           'pavement',
           'street_light',
           'bike_lanes',
           'street_amenity',
           'utility_pole',
           'bike_parking',
           'side_walk',
           'cross_walk',
           'accessibility'
          ]

for city in city_list:
    # load location
    location=os.path.join(root,'data/meta_data_{}/{}_remaining_points.csv'.format(city,city))
    location_df=pd.read_csv(location)
    location_df['panoId']='pano='+location_df['panoId']
    location_df=location_df.rename(columns={'panoId':'pano_id'})
    
    # go through data_list and left join it to location_df
    for data in data_list:
        # import as df
        data_df=pd.read_csv(os.path.join(root,'data/tabular_data/{}/{}.csv'.format(city,data)))
        data_df=data_df.iloc[:,1:]
        data_df=data_df.rename(columns={data_df.columns[0]:'pano_id',data_df.columns[-1]:data})
    
        # merge and scale them
        location_df=pd.merge(location_df,
                         data_df,
                         on='pano_id',
                         how='left'
                        ) 
        location_df.to_csv(os.path.join(root,'data/tabular_data/{}/only_svi_infrastructure_before_scaling.csv'.format(city)))
        
# scaling
# put both cities data together first
df_temp=pd.DataFrame()
for city in city_list:
    infrastructure_df=pd.read_csv(os.path.join(root,'data/tabular_data/{}/only_svi_infrastructure_before_scaling.csv'.format(city)))
    infrastructure_df['city']=city
    df_temp=df_temp.append(infrastructure_df)
# 1 if >0
cols_transfomation_above_0=['street_light','bike_lanes','street_amenity','bike_parking',
                            'side_walk','cross_walk','accessibility']
for col in cols_transfomation_above_0:
    cond_list=[df_temp[col]>0,df_temp[col]==0]
    choice_list=[1,0]
    df_temp[col]=np.select(cond_list,choice_list)
# 1 if ==0
cols_transfomation_equal_to_0=['pavement','utility_pole']
for col in cols_transfomation_equal_to_0:
    cond_list=[df_temp[col]==0,df_temp[col]>0]
    choice_list=[1,0]
    df_temp[col]=np.select(cond_list,choice_list)

# calculate the indicator
df_temp['infrastructure']=df_temp[data_list].sum(axis=1)*((100/num_category)/len(data_list))
# save seperately by cities
for city in city_list:
    df_temp_city=df_temp.loc[df_temp['city']==city]
    print(df_temp_city)
    df_temp_city.to_csv(os.path.join(root,'data/tabular_data/{}/only_svi_infrastructure.csv'.format(city)))

      Unnamed: 0  Unnamed: 0.1  \
0              0             0   
1              1             1   
2              2             2   
3              3             3   
4              4             4   
...          ...           ...   
5828        5828          6115   
5829        5829          6116   
5830        5830          6117   
5831        5831          6119   
5832        5832          6120   

                                                pano_id     panoLon   panoLat  \
0                           pano=EIzSniBOXDgT4m2fGH2SNQ  103.914512  1.400917   
1                           pano=tzvazBY0ag5RgAsvJgHvZA  103.965906  1.375535   
2                           pano=FJjCz3jIAWYYnGq4PvR9HA  103.819038  1.293706   
3                           pano=TfQN73pv-6mAZy2QnMIAPA  103.703747  1.341976   
4                           pano=PATYQYa4O43_LjJ_JMbrFw  103.900929  1.311461   
...                                                 ...         ...       ...   
5828  pano=CAoSLEFGMVFpc

### Vehicle_Cyclist_Interaction
- No. of vehicles
- traffic light / stop sign

In [15]:
num_category=4
city_list=['Singapore',
           'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"

# min-max scaler
scaler = MinMaxScaler()

data_list=['no_of_vehicles',
           'traffic_light_stop_sign'
          ]

for city in city_list:
    # load location
    location=os.path.join(root,'data/meta_data_{}/{}_remaining_points.csv'.format(city,city))
    location_df=pd.read_csv(location)
    location_df['panoId']='pano='+location_df['panoId']
    location_df=location_df.rename(columns={'panoId':'pano_id'})
    
    # go through data_list and left join it to location_df
    for data in data_list:
        # import as df
        data_df=pd.read_csv(os.path.join(root,'data/tabular_data/{}/{}.csv'.format(city,data)))
        data_df=data_df.iloc[:,1:]
        data_df=data_df.rename(columns={data_df.columns[0]:'pano_id',data_df.columns[-1]:data})
    
        # merge and scale them
        location_df=pd.merge(location_df,
                         data_df,
                         on='pano_id',
                         how='left'
                        ) 
        location_df.to_csv(os.path.join(root,'data/tabular_data/{}/only_svi_vehicle_cyclist_interaction_before_scaling.csv'.format(city)))
        
# scaling
# put both cities data together first
df_temp=pd.DataFrame()
for city in city_list:
    vehicle_cyclist_interaction_df=pd.read_csv(os.path.join(root,'data/tabular_data/{}/only_svi_vehicle_cyclist_interaction_before_scaling.csv'.format(city)))
    vehicle_cyclist_interaction_df['city']=city
    df_temp=df_temp.append(vehicle_cyclist_interaction_df)

# scaling using Min-Max scale
cols_transfomation_negative=['no_of_vehicles']
df_temp[cols_transfomation_negative]=scaler.fit_transform(df_temp[cols_transfomation_negative])
df_temp[cols_transfomation_negative]=1-df_temp[cols_transfomation_negative]


# calculate the indicator
df_temp['vehicle_cyclist_interaction']=df_temp[data_list].sum(axis=1)*((100/num_category)/len(data_list))
# save seperately by cities
for city in city_list:
    df_temp_city=df_temp.loc[df_temp['city']==city]
    print(df_temp_city)
    df_temp_city.to_csv(os.path.join(root,'data/tabular_data/{}/only_svi_vehicle_cyclist_interaction.csv'.format(city)))

      Unnamed: 0  Unnamed: 0.1  \
0              0             0   
1              1             1   
2              2             2   
3              3             3   
4              4             4   
...          ...           ...   
5828        5828          6115   
5829        5829          6116   
5830        5830          6117   
5831        5831          6119   
5832        5832          6120   

                                                pano_id     panoLon   panoLat  \
0                           pano=EIzSniBOXDgT4m2fGH2SNQ  103.914512  1.400917   
1                           pano=tzvazBY0ag5RgAsvJgHvZA  103.965906  1.375535   
2                           pano=FJjCz3jIAWYYnGq4PvR9HA  103.819038  1.293706   
3                           pano=TfQN73pv-6mAZy2QnMIAPA  103.703747  1.341976   
4                           pano=PATYQYa4O43_LjJ_JMbrFw  103.900929  1.311461   
...                                                 ...         ...       ...   
5828  pano=CAoSLEFGMVFpc

## Perception
- beauty
- building_attractiveness
- cleanliness
- cycling_attractiveness
- living_attractiveness
- safety
- spaciousness

In [16]:
num_category=4
city_list=['Singapore',
           'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"


data_list=['beauty',
          'building_attractiveness','cleanliness','cycling_attractiveness',
          'living_attractiveness','safety','spaciousness'
          ]

for city in city_list:
    # load location
    location=os.path.join(root,'data/meta_data_{}/{}_remaining_points.csv'.format(city,city))
    location_df=pd.read_csv(location)
    location_df['panoId']='pano='+location_df['panoId']
    location_df=location_df.rename(columns={'panoId':'pano_id'})
    
    # go through data_list and left join it to location_df
    for data in data_list:
        # import as df
        data_df=pd.read_csv(os.path.join(root,'data/tabular_data/{}/predicted_{}.csv'.format(city,data)))
        data_df=data_df.iloc[:,1:]
        # merge and scale them
        location_df=pd.merge(location_df,
                         data_df,
                         on='pano_id',
                         how='left'
                        )
    # scaling
    location_df = location_df.apply(lambda x: x/10 if x.name.startswith('predicted_') else x)
    location_df['perception']=location_df[location_df.columns[pd.Series(location_df.columns).str.startswith('predicted_')]].\
        sum(axis=1)*((100/num_category)/len(data_list))
    print(location_df)
    location_df.to_csv(os.path.join(root,'data/tabular_data/{}/only_svi_perception.csv'.format(city)))

      Unnamed: 0                                            pano_id  \
0              0                        pano=EIzSniBOXDgT4m2fGH2SNQ   
1              1                        pano=tzvazBY0ag5RgAsvJgHvZA   
2              2                        pano=FJjCz3jIAWYYnGq4PvR9HA   
3              3                        pano=TfQN73pv-6mAZy2QnMIAPA   
4              4                        pano=PATYQYa4O43_LjJ_JMbrFw   
...          ...                                                ...   
5828        6115  pano=CAoSLEFGMVFpcE9YcGFGMVpBOURjN21NS0NQS1h3L...   
5829        6116                        pano=4I7X92FmvGD7ALNXMZQuAg   
5830        6117                        pano=EghtrEJcRO-l7XlA21zaNQ   
5831        6119                        pano=O6UhfdrDWVrKDmJTaOLmFA   
5832        6120                        pano=uvufId4xvxyRl-4TUehx_A   

         panoLon   panoLat  distDiff  predicted_beauty  \
0     103.914512  1.400917  0.000084          0.677530   
1     103.965906  1.375535  0.0

### Bikeability
- environment
- infrastructure
- vehicle cyclist interaction
- perception

In [17]:
num_category=4
city_list=['Singapore',
           'Tokyo'
          ]
root = "/home/ec2-user/SageMaker/"


data_list=[
           'environment',
           'infrastructure',
           'vehicle_cyclist_interaction',
           'perception'
          ]

for city in city_list:
    # load location
    location=os.path.join(root,'data/meta_data_{}/{}_remaining_points.csv'.format(city,city))
    location_df=pd.read_csv(location)
    location_df['panoId']='pano='+location_df['panoId']
    location_df=location_df.rename(columns={'panoId':'pano_id'})
    
    # go through data_list and left join it to location_df
    for data in data_list:
        # import as df
        data_df=pd.read_csv(os.path.join(root,'data/tabular_data/{}/only_svi_{}.csv'.format(city,data)))
        data_df=data_df.iloc[:,1:]
        # merge and scale them
        location_df=pd.merge(location_df,
                         data_df[['pano_id',data]],
                         on='pano_id',
                         how='left'
                        )
        location_df.drop(location_df.filter(regex='Unnamed').columns, axis=1, inplace=True)
        
    # scaling
    location_df['bikeability']=location_df.iloc[:,-(num_category):].sum(axis=1)
    print(location_df)
    location_df.to_csv(os.path.join(root,'data/tabular_data/{}/only_svi_bikeability.csv'.format(city)))

                                                pano_id     panoLon   panoLat  \
0                           pano=EIzSniBOXDgT4m2fGH2SNQ  103.914512  1.400917   
1                           pano=tzvazBY0ag5RgAsvJgHvZA  103.965906  1.375535   
2                           pano=FJjCz3jIAWYYnGq4PvR9HA  103.819038  1.293706   
3                           pano=TfQN73pv-6mAZy2QnMIAPA  103.703747  1.341976   
4                           pano=PATYQYa4O43_LjJ_JMbrFw  103.900929  1.311461   
...                                                 ...         ...       ...   
5828  pano=CAoSLEFGMVFpcE9YcGFGMVpBOURjN21NS0NQS1h3L...  103.835671  1.289808   
5829                        pano=4I7X92FmvGD7ALNXMZQuAg  103.966108  1.373809   
5830                        pano=EghtrEJcRO-l7XlA21zaNQ  103.701994  1.344878   
5831                        pano=O6UhfdrDWVrKDmJTaOLmFA  103.632836  1.324383   
5832                        pano=uvufId4xvxyRl-4TUehx_A  103.747407  1.428744   

      distDiff  environment